<a href="https://colab.research.google.com/github/gui98araujo/nflAnalytics/blob/main/Rede_Neural_Bets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Apostas Esportivas

Nesse projeto, o objetivo será obter odds esportivas de acordo com algum modelo de machine learning de classificação que calcule a probabilidade de cada evento esportivo acontecer.
Com o intuito de simplificar os estudos, o objeto de estudo desse projeto será os resultados de jogos de ligas esportivas no formato de pontos corridos, pois, como nesse formato todas as equipes jogam entre si, será possível então determinar modelos mais intuitivos que consigam corresponder as variáveis que o autor planeja criar no processo de features engenering.
Os dados foram obtidos através de um web Scraping através do site BetExplorer e foram previamente tratados em outro arquivo.

In [ ]:
import pandas as pd
import numpy as np

# Importar o conjunto de dados
df = pd.read_excel("/content/BRA.xlsx")

df2012=df[df['Season']==2012]
# Ordenar o conjunto de dados pela data dos jogos

In [ ]:
df2012

,Country,League,Season,Date,Time,Home,Away,HG,AG,Res,PH,PD,PA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA
0,Brazil,Serie A,2012,2012-05-19,22:30:00,Palmeiras,Portuguesa,1,1,D,1.75,3.86,5.25,1.76,3.87,5.31,1.69,3.50,4.90
1,Brazil,Serie A,2012,2012-05-19,22:30:00,Sport Recife,Flamengo RJ,1,1,D,2.83,3.39,2.68,2.83,3.42,2.7,2.59,3.23,2.58
2,Brazil,Serie A,2012,2012-05-20,01:00:00,Figueirense,Nautico,2,1,H,1.6,4.04,6.72,1.67,4.05,7.22,1.59,3.67,5.64
3,Brazil,Serie A,2012,2012-05-20,20:00:00,Botafogo RJ,Sao Paulo,4,2,H,2.49,3.35,3.15,2.49,3.39,3.15,2.35,3.26,2.84
4,Brazil,Serie A,2012,2012-05-20,20:00:00,Corinthians,Fluminense,0,1,A,1.96,3.53,4.41,1.96,3.53,4.41,1.89,3.33,3.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,Brazil,Serie A,2012,2012-12-02,19:00:00,Fluminense,Vasco,1,2,A,2.39,3.31,3.28,2.46,3.45,3.3,2.27,3.26,2.99
376,Brazil,Serie A,2012,2012-12-02,19:00:00,Gremio,Internacional,0,0,D,1.67,3.93,5.78,1.7,4.25,6,1.64,3.68,4.97
377,Brazil,Serie A,2012,2012-12-02,19:00:00,Nautico,Sport Recife,1,0,H,2.72,3.55,2.66,2.73,3.59,2.95,2.49,3.30,2.65
378,Brazil,Serie A,2012,2012-12-02,19:00:00,Portuguesa,Ponte Preta,0,0,D,1.8,3.57,5.25,1.85,3.6,5.4,1.77,3.37,4.56


In [ ]:

# Converter as datas para o tipo datetime
df2012['Date'] = pd.to_datetime(df2012['Date'], format='%d/%m/%Y')


# Ordenar o dataset por data
df2012 = df2012.sort_values('Date')

# Definir a função para calcular a quantidade de pontos do time nas últimas 10 rodadas
def last_10_points(team, date):
    last_10 = df2012[(df2012['Date'] < date) & ((df2012['Home'] == team) | (df2012['Away'] == team))][-10:]
    points = sum([3 if ((row['Home'] == team and row['Res'] == 'H') or (row['Away'] == team and row['Res'] == 'A')) else (1 if row['Res'] == 'D' else 0) for _, row in last_10.iterrows()])
    return points

# Calcular a quantidade de pontos de cada time nas últimas 10 rodadas antes de cada jogo
df2012['points_home'] = df2012.apply(lambda row: last_10_points(row['Home'], row['Date']), axis=1)
df2012['points_away'] = df2012.apply(lambda row: last_10_points(row['Away'], row['Date']), axis=1)


<ipython-input-10-a83dafc85d12>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2012['Date'] = pd.to_datetime(df2012['Date'], format='%d/%m/%Y')


In [ ]:
df2012.tail(10)

,Country,League,Season,Date,Time,Home,Away,HG,AG,Res,...,PD,PA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,points_home,points_away
370,Brazil,Serie A,2012,2012-12-01,21:30:00,Flamengo RJ,Botafogo RJ,2,2,D,...,3.38,3.18,2.54,3.38,3.2,2.33,3.22,2.90,15,14
371,Brazil,Serie A,2012,2012-12-01,21:30:00,Santos,Palmeiras,3,1,H,...,4.55,5.91,1.65,4.59,6,1.57,3.89,5.33,16,8
376,Brazil,Serie A,2012,2012-12-02,19:00:00,Gremio,Internacional,0,0,D,...,3.93,5.78,1.7,4.25,6,1.64,3.68,4.97,20,10
377,Brazil,Serie A,2012,2012-12-02,19:00:00,Nautico,Sport Recife,1,0,H,...,3.55,2.66,2.73,3.59,2.95,2.49,3.30,2.65,12,14
375,Brazil,Serie A,2012,2012-12-02,19:00:00,Fluminense,Vasco,1,2,A,...,3.31,3.28,2.46,3.45,3.3,2.27,3.26,2.99,18,8
378,Brazil,Serie A,2012,2012-12-02,19:00:00,Portuguesa,Ponte Preta,0,0,D,...,3.57,5.25,1.85,3.6,5.4,1.77,3.37,4.56,11,13
373,Brazil,Serie A,2012,2012-12-02,19:00:00,Atletico-MG,Cruzeiro,3,2,H,...,4.25,4.97,1.75,4.3,5.2,1.68,3.70,4.60,16,16
372,Brazil,Serie A,2012,2012-12-02,19:00:00,Atletico GO,Bahia,0,1,A,...,3.31,2.37,3.27,3.4,2.41,3.01,3.18,2.28,10,10
374,Brazil,Serie A,2012,2012-12-02,19:00:00,Coritiba,Figueirense,3,0,H,...,5.18,8.82,1.4,5.2,9.69,1.36,4.62,7.80,16,8
379,Brazil,Serie A,2012,2012-12-02,19:00:00,Sao Paulo,Corinthians,3,1,H,...,3.46,2.25,3.4,3.5,2.38,3.07,3.23,2.23,20,18


In [ ]:
df2012['perc_points_home_team'] = df2012['points_home'] / 30
df2012['perc_points_away_team'] = df2012['points_away'] / 30

df2012 = df2012.drop(['points_home', 'points_away'], axis=1)

In [ ]:
# Definir nível dos times no início do campeonato
team_levels_2012 = {
    "Corinthians": 5,
    "Vasco": 4.5,
    "Fluminense": 4,
    "Botafogo RJ": 4,
    "Santos": 4,
    "Sao Paulo": 4,
    "Internacional": 3.5,
    "Cruzeiro": 3.5,
    "Atletico-MG": 3.5,
    "Gremio": 3.5,
    "Palmeiras": 3,
    "Figueirense": 3,
    "Flamengo RJ": 3,
    "Bahia": 3,
    "Portuguesa": 2.5,
    "Ponte Preta": 2.5,
    "Nautico": 2.5,
    "Sport Recife": 2.5,
    "Atletico GO": 2,
    "Coritiba": 2,
}

# Atualizar nível dos times após a décima nona rodada
team_levels_2012_rodada19 = {
    "Corinthians": 5,
    "Atletico-MG": 4.5,
    "Gremio": 4.5,
    "Vasco": 4,
    "Botafogo RJ": 4,
    "Fluminense": 4,
    "Sao Paulo": 4,
    "Internacional": 4,
    "Cruzeiro": 4,
    "Nautico": 3.5,
    "Ponte Preta": 3.5,
    "Flamengo RJ": 3.5,
    "Santos": 3.5,
    "Portuguesa": 3,
    "Atletico GO": 3,
    "Coritiba": 3,
    "Sport Recife": 2.5,
    "Bahia": 2.5,
    "Figueirense": 2.5,
    "Palmeiras": 2.5,
}



In [ ]:
# Criar lista de times que aparecem no dataset
team_list = list(set(df2012['Home'].unique()) | set(df2012['Away'].unique()))

# Criar colunas de níveis dos times em cada partida
df2012['level_team_home'] = 0
df2012['level_team_away'] = 0

# Preencher colunas com níveis dos times
for team in team_list:
    # Preencher com nível até a 19ª aparição do time no dataset
    team_rows = df2012.loc[((df2012['Home'] == team) | (df2012['Away'] == team))].iloc[:19]
    df2012.loc[((df2012['Home'] == team) & (df2012.index.isin(team_rows.index))), 'level_team_home'] = team_levels_2012[team]
    df2012.loc[((df2012['Away'] == team) & (df2012.index.isin(team_rows.index))), 'level_team_away'] = team_levels_2012[team]
    # Preencher com nível a partir da 20ª aparição do time no dataset
    team_rows = df2012.loc[((df2012['Home'] == team) | (df2012['Away'] == team))].iloc[19:]
    df2012.loc[((df2012['Home'] == team) & (df2012.index.isin(team_rows.index))), 'level_team_home'] = team_levels_2012_rodada19[team]
    df2012.loc[((df2012['Away'] == team) & (df2012.index.isin(team_rows.index))), 'level_team_away'] = team_levels_2012_rodada19[team]


In [ ]:
df2012.head(10)

,Country,League,Season,Date,Time,Home,Away,HG,AG,Res,...,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,perc_points_home_team,perc_points_away_team,level_team_home,level_team_away
0,Brazil,Serie A,2012,2012-05-19,22:30:00,Palmeiras,Portuguesa,1,1,D,...,1.76,3.87,5.31,1.69,3.50,4.90,0.0,0.0,3.0,2.5
1,Brazil,Serie A,2012,2012-05-19,22:30:00,Sport Recife,Flamengo RJ,1,1,D,...,2.83,3.42,2.7,2.59,3.23,2.58,0.0,0.0,2.5,3.0
2,Brazil,Serie A,2012,2012-05-20,01:00:00,Figueirense,Nautico,2,1,H,...,1.67,4.05,7.22,1.59,3.67,5.64,0.0,0.0,3.0,2.5
3,Brazil,Serie A,2012,2012-05-20,20:00:00,Botafogo RJ,Sao Paulo,4,2,H,...,2.49,3.39,3.15,2.35,3.26,2.84,0.0,0.0,4.0,4.0
4,Brazil,Serie A,2012,2012-05-20,20:00:00,Corinthians,Fluminense,0,1,A,...,1.96,3.53,4.41,1.89,3.33,3.89,0.0,0.0,5.0,4.0
5,Brazil,Serie A,2012,2012-05-20,20:00:00,Internacional,Coritiba,2,0,H,...,1.77,3.9,6,1.66,3.55,5.10,0.0,0.0,3.5,2.0
6,Brazil,Serie A,2012,2012-05-20,20:00:00,Ponte Preta,Atletico-MG,0,1,A,...,3.12,3.45,2.62,2.74,3.23,2.45,0.0,0.0,2.5,3.5
7,Brazil,Serie A,2012,2012-05-20,22:30:00,Bahia,Santos,0,0,D,...,2.45,3.52,3.32,2.30,3.22,2.97,0.0,0.0,3.0,4.0
8,Brazil,Serie A,2012,2012-05-20,22:30:00,Cruzeiro,Atletico GO,0,0,D,...,1.85,3.79,4.82,1.78,3.47,4.30,0.0,0.0,3.5,2.0
9,Brazil,Serie A,2012,2012-05-20,22:30:00,Vasco,Gremio,2,1,H,...,2.25,3.46,3.64,2.08,3.25,3.41,0.0,0.0,4.5,3.5


In [ ]:
df2012.tail(10)

,Country,League,Season,Date,Time,Home,Away,HG,AG,Res,...,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,perc_points_home_team,perc_points_away_team,level_team_home,level_team_away
370,Brazil,Serie A,2012,2012-12-01,21:30:00,Flamengo RJ,Botafogo RJ,2,2,D,...,2.54,3.38,3.2,2.33,3.22,2.90,0.500000,0.466667,3.5,4.0
371,Brazil,Serie A,2012,2012-12-01,21:30:00,Santos,Palmeiras,3,1,H,...,1.65,4.59,6,1.57,3.89,5.33,0.533333,0.266667,3.5,2.5
376,Brazil,Serie A,2012,2012-12-02,19:00:00,Gremio,Internacional,0,0,D,...,1.7,4.25,6,1.64,3.68,4.97,0.666667,0.333333,4.5,4.0
377,Brazil,Serie A,2012,2012-12-02,19:00:00,Nautico,Sport Recife,1,0,H,...,2.73,3.59,2.95,2.49,3.30,2.65,0.400000,0.466667,3.5,2.5
375,Brazil,Serie A,2012,2012-12-02,19:00:00,Fluminense,Vasco,1,2,A,...,2.46,3.45,3.3,2.27,3.26,2.99,0.600000,0.266667,4.0,4.0
378,Brazil,Serie A,2012,2012-12-02,19:00:00,Portuguesa,Ponte Preta,0,0,D,...,1.85,3.6,5.4,1.77,3.37,4.56,0.366667,0.433333,3.0,3.5
373,Brazil,Serie A,2012,2012-12-02,19:00:00,Atletico-MG,Cruzeiro,3,2,H,...,1.75,4.3,5.2,1.68,3.70,4.60,0.533333,0.533333,4.5,4.0
372,Brazil,Serie A,2012,2012-12-02,19:00:00,Atletico GO,Bahia,0,1,A,...,3.27,3.4,2.41,3.01,3.18,2.28,0.333333,0.333333,3.0,2.5
374,Brazil,Serie A,2012,2012-12-02,19:00:00,Coritiba,Figueirense,3,0,H,...,1.4,5.2,9.69,1.36,4.62,7.80,0.533333,0.266667,3.0,2.5
379,Brazil,Serie A,2012,2012-12-02,19:00:00,Sao Paulo,Corinthians,3,1,H,...,3.4,3.5,2.38,3.07,3.23,2.23,0.666667,0.600000,4.0,5.0


In [ ]:
# Definir a função para calcular a quantidade de pontos do time nos últimos 7 jogos em casa/fora de casa
def last_7_points(team, date, location):
    last_7 = df2012[(df2012['Date'] < date) & (((df2012['Home'] == team) & (location == 'home')) | ((df2012['Away'] == team) & (location == 'away')))][-7:]
    points = sum([3 if ((row['Home'] == team and row['Res'] == 'H') or (row['Away'] == team and row['Res'] == 'A')) else (1 if row['Res'] == 'D' else 0) for _, row in last_7.iterrows()])
    return points/21  # dividir por 21 para obter a média de pontos por jogo

# Calcular a quantidade de pontos dos últimos 7 jogos em casa/fora de casa de cada time antes de cada jogo
df2012['last_7_home'] = df2012.apply(lambda row: last_7_points(row['Home'], row['Date'], 'home'), axis=1)
df2012['last_7_away'] = df2012.apply(lambda row: last_7_points(row['Away'], row['Date'], 'away'), axis=1)


In [ ]:
df2012

,Country,League,Season,Date,Time,Home,Away,HG,AG,Res,...,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,perc_points_home_team,perc_points_away_team,last_7_home,last_7_away
0,Brazil,Serie A,2012,2012-05-19,22:30:00,Palmeiras,Portuguesa,1,1,D,...,1.76,3.87,5.31,1.69,3.50,4.90,0.000000,0.000000,0.000000,0.000000
1,Brazil,Serie A,2012,2012-05-19,22:30:00,Sport Recife,Flamengo RJ,1,1,D,...,2.83,3.42,2.7,2.59,3.23,2.58,0.000000,0.000000,0.000000,0.000000
2,Brazil,Serie A,2012,2012-05-20,01:00:00,Figueirense,Nautico,2,1,H,...,1.67,4.05,7.22,1.59,3.67,5.64,0.000000,0.000000,0.000000,0.000000
3,Brazil,Serie A,2012,2012-05-20,20:00:00,Botafogo RJ,Sao Paulo,4,2,H,...,2.49,3.39,3.15,2.35,3.26,2.84,0.000000,0.000000,0.000000,0.000000
4,Brazil,Serie A,2012,2012-05-20,20:00:00,Corinthians,Fluminense,0,1,A,...,1.96,3.53,4.41,1.89,3.33,3.89,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,Brazil,Serie A,2012,2012-12-02,19:00:00,Portuguesa,Ponte Preta,0,0,D,...,1.85,3.6,5.4,1.77,3.37,4.56,0.366667,0.433333,0.333333,0.047619
373,Brazil,Serie A,2012,2012-12-02,19:00:00,Atletico-MG,Cruzeiro,3,2,H,...,1.75,4.3,5.2,1.68,3.70,4.60,0.533333,0.533333,0.714286,0.190476
372,Brazil,Serie A,2012,2012-12-02,19:00:00,Atletico GO,Bahia,0,1,A,...,3.27,3.4,2.41,3.01,3.18,2.28,0.333333,0.333333,0.285714,0.285714
374,Brazil,Serie A,2012,2012-12-02,19:00:00,Coritiba,Figueirense,3,0,H,...,1.4,5.2,9.69,1.36,4.62,7.80,0.533333,0.266667,0.619048,0.142857


# Pontos gerais

In [ ]:
team_points = {}

df['points_Home'] = 0
df['points_Away'] = 0

for index, row in df.iterrows():
    home_team = row['Home']
    away_team = row['Away']
    result = row['Result']

    if home_team not in team_points:
        team_points[home_team] = 0
    if away_team not in team_points:
        team_points[away_team] = 0

    # Calcula as pontuações acumuladas sem contar com a pontuação da linha atual
    points_home = team_points[home_team]
    points_away = team_points[away_team]

    if result == 'H':
        points_home += 3
    elif result == 'D':
        points_home += 1
        points_away += 1
    elif result == 'A':
        points_away += 3

    df.at[index, 'points_Home'] = points_home
    df.at[index, 'points_Away'] = points_away

    # Atualiza a pontuação acumulada para as próximas rodadas
    team_points[home_team] = points_home
    team_points[away_team] = points_away


df['points_h'] = 0
df['points_a'] = 0

for index, row in df.iterrows():
    if row['Result'] == 'H':
        df.at[index, 'points_h'] = row['points_Home'] - 3
        df.at[index, 'points_a'] = row['points_Away']
    elif row['Result'] == 'A':
        df.at[index, 'points_h'] = row['points_Home']
        df.at[index, 'points_a'] = row['points_Away'] - 3
    elif row['Result'] == 'D':
        df.at[index, 'points_h'] = row['points_Home'] - 1
        df.at[index, 'points_a'] = row['points_Away'] - 1

In [ ]:
team_levels_2013 = {
"Atletico-MG": 5,
"Corinthians": 5,
"Fluminense": 4.5,
"Gremio": 4.5,
"Sao Paulo": 4.5,
"Cruzeiro": 4,
"Internacional": 4,
"Botafogo RJ": 4,
"Santos": 4,
"Vasco": 3.5,
"Portuguesa": 3,
"Coritiba": 3,
"Bahia": 3,
"Ponte Preta": 3,
"Flamengo RJ": 3,
"Nautico": 2.5,
"Goias": 2.5,
"Criciuma": 2.5,
"Vitoria": 2.5,
"Atletico-PR": 3,
}

# Níveis dos times a partir da 20ª rodada
team_levels_2013_rodada19 = {
"Cruzeiro": 5,
"Botafogo RJ": 4.5,
"Gremio": 4.5,
"Sao Paulo": 4.5,
"Corinthians": 4.5,
"Fluminense": 3.5,
"Internacional": 4,
"Atletico-MG": 4,
"Santos": 4,
"Vitoria": 3.5,
"Goias": 3.5,
"Coritiba": 3.5,
"Bahia": 3,
"Flamengo RJ": 3,
"Portuguesa": 3,
"Nautico": 1.5,
"Criciuma": 2.5,
"Ponte Preta": 2.5,
"Atletico-PR": 4,
"Vasco": 3,
}

team_levels_2014 = {
    "Cruzeiro": 5,
    "Sao Paulo": 4.5,
    "Internacional": 4.5,
    "Atletico-MG": 4.5,
    "Fluminense": 4.5,
    "Gremio": 4,
    "Corinthians": 4,
    "Santos": 4,
    "Atletico-PR": 3.5,
    "Botafogo RJ": 3.5,
    "Goias": 3.5,
    "Palmeiras": 3.5,
    "Flamengo RJ": 3,
    "Vitoria": 3,
    "Sport Recife": 3,
    "Chapecoense-SC": 3,
    "Figueirense": 2.5,
    "Coritiba": 2.5,
    "Bahia": 2.5,
    "Criciuma": 2,
}

team_levels_2014_rodada19 = {
    "Cruzeiro": 5,
    "Sao Paulo": 4.5,
    "Internacional": 4.5,
    "Atletico-MG": 4.5,
    "Fluminense": 4.5,
    "Gremio": 4,
    "Corinthians": 4,
    "Santos": 4,
    "Atletico-PR": 3.5,
    "Botafogo RJ": 3.5,
    "Goias": 3.5,
    "Palmeiras": 3.5,
    "Flamengo RJ": 3,
    "Vitoria": 3,
    "Sport Recife": 3,
    "Chapecoense-SC": 3,
    "Figueirense": 2.5,
    "Coritiba": 2.5,
    "Bahia": 2.5,
    "Criciuma": 2,
}


team_levels_2015 = {
    "Corinthians": 5,
    "Atletico-MG": 4.5,
    "Gremio": 4.5,
    "Palmeiras": 4,
    "Santos": 4,
    "Fluminense": 4,
    "Cruzeiro": 4,
    "Sao Paulo": 4,
    "Internacional": 3.5,
    "Sport Recife": 3.5,
    "Atletico-PR": 3.5,
    "Ponte Preta": 3.5,
    "Chapecoense-SC": 3,
    "Coritiba": 3,
    "Flamengo RJ": 3,
    "Vasco": 3,
    "Avai": 2.5,
    "Goias": 2.5,
    "Joinville": 2.5,
    "Figueirense": 2,
}

team_levels_2015_rodada19 = {
    "Corinthians": 5,
    "Atletico-MG": 4.5,
    "Gremio": 4.5,
    "Palmeiras": 4,
    "Santos": 4,
    "Fluminense": 4,
    "Cruzeiro": 4,
    "Sao Paulo": 4,
    "Internacional": 3.5,
    "Sport Recife": 3.5,
    "Atletico-PR": 3.5,
    "Ponte Preta": 3.5,
    "Chapecoense-SC": 3,
    "Coritiba": 3,
    "Flamengo RJ": 3,
    "Vasco": 3,
    "Avai": 2.5,
    "Goias": 2.5,
    "Joinville": 2.5,
    "Figueirense": 2,
}

team_levels_2016 = {
    "Palmeiras": 5,
    "Santos": 4.5,
    "Flamengo RJ": 4.5,
    "Atletico-MG": 4.5,
    "Botafogo RJ": 4,
    "Atletico-PR": 4,
    "Corinthians": 4,
    "Gremio": 4,
    "Fluminense": 3.5,
    "Ponte Preta": 3.5,
    "Chapecoense-SC": 3.5,
    "Sao Paulo": 3.5,
    "Cruzeiro": 3,
    "Sport Recife": 3,
    "Vitoria": 3,
    "Coritiba": 3,
    "Internacional": 2.5,
    "Figueirense": 2.5,
    "Santa Cruz": 2.5,
    "America MG": 2,
}

team_levels_2016_rodada19 = {
    "Palmeiras": 5,
    "Santos": 4.5,
    "Flamengo RJ": 4.5,
    "Atletico-MG": 4.5,
    "Botafogo RJ": 4,
    "Atletico-PR": 4,
    "Corinthians": 4,
    "Gremio": 4,
    "Fluminense": 3.5,
    "Ponte Preta": 3.5,
    "Chapecoense-SC": 3.5,
    "Sao Paulo": 3.5,
    "Cruzeiro": 3.5,
    "Sport Recife": 3.5,
    "Vitoria": 3,
    "Coritiba": 3,
    "Internacional": 2.5,
    "Figueirense": 2.5,
    "Santa Cruz": 1.5,
    "America MG": 2,
}

team_levels_2018 = {
    "Palmeiras": 5,
    "Flamengo RJ": 5,
    "Internacional": 4.5,
    "Gremio": 4.5,
    "Sao Paulo": 4.5,
    "Atletico-MG": 4.5,
    "Atletico-PR": 4,
    "Cruzeiro": 4,
    "Botafogo RJ": 3.5,
    "Santos": 4,
    "Bahia": 3.5,
    "Corinthians": 3.5,
    "Vasco": 3,
    "Fluminense": 3,
    "Sport Recife": 3,
    "Ceara": 3,
    "Chapecoense-SC": 3,
    "Vitoria": 2.5,
    "Parana": 2.5,
    "America MG": 2
}

team_levels_2018_rodada19 = {
    "Palmeiras": 5,
    "Flamengo RJ": 5,
    "Internacional": 4.5,
    "Gremio": 4.5,
    "Sao Paulo": 4.5,
    "Atletico-MG": 4.5,
    "Atletico-PR": 4,
    "Cruzeiro": 4,
    "Botafogo RJ": 4,
    "Santos": 4,
    "Bahia": 3.5,
    "Corinthians": 3.5,
    "Vasco": 3,
    "Fluminense": 3,
    "Sport Recife": 3,
    "Ceara": 3,
    "Chapecoense-SC": 3,
    "Vitoria": 2.5,
    "Parana": 2.5,
    "America MG": 2
}

team_levels_2019 = {
    "Flamengo RJ": 5,
    "Palmeiras": 4.5,
    "Santos": 4.5,
    "Gremio": 4.5,
    "Atletico-MG": 4.5,
    "Sao Paulo": 4,
    "Internacional": 4,
    "Corinthians": 4,
    "Bahia": 3.5,
    "Goias": 3.5,
    "Athletico-PR": 3.5,
    "Botafogo RJ": 3,
    "Vasco": 3,
    "Fluminense": 3,
    "Ceara": 3,
    "Fortaleza": 3,
    "Cruzeiro": 2.5,
    "CSA": 2.5,
    "Chapecoense-SC": 2.5,
    "Avai": 2
}

team_levels_2019_rodada19 = {
    "Flamengo RJ": 5,
    "Palmeiras": 4.5,
    "Santos": 4.5,
    "Gremio": 4.5,
    "Atletico-MG": 4.5,
    "Sao Paulo": 4,
    "Internacional": 4,
    "Corinthians": 4,
    "Bahia": 3.5,
    "Goias": 3.5,
    "Athletico-PR": 3.5,
    "Botafogo RJ": 3,
    "Vasco": 3,
    "Fluminense": 3,
    "Ceara": 3,
    "Fortaleza": 3,
    "Cruzeiro": 2.5,
    "CSA": 2.5,
    "Chapecoense-SC": 2.5,
    "Avai": 2
}


In [ ]:
df2013 = df[df['Season'] == 2013]

# Convert dates to datetime type
df2013['Date'] = pd.to_datetime(df2013['Date'], format='%d/%m/%Y')

# Sort the dataset by date
df2013 = df2013.sort_values('Date')

# Define the function to calculate the number of points for the team in the last 10 rounds
def last_10_points(team, date):
    last_10 = df2013[(df2013['Date'] < date) & ((df2013['Home'] == team) | (df2013['Away'] == team))][-10:]
    points = sum([3 if ((row['Home'] == team and row['Res'] == 'H') or (row['Away'] == team and row['Res'] == 'A')) else (1 if row['Res'] == 'D' else 0) for _, row in last_10.iterrows()])
    return points

# Calculate the number of points for each team in the last 10 rounds before each game
df2013['points_home'] = df2013.apply(lambda row: last_10_points(row['Home'], row['Date']), axis=1)
df2013['points_away'] = df2013.apply(lambda row: last_10_points(row['Away'], row['Date']), axis=1)

df2013['perc_points_home_team'] = df2013['points_home'] / 30
df2013['perc_points_away_team'] = df2013['points_away'] / 30

df2013 = df2013.drop(['points_home', 'points_away'], axis=1)

# Create a list of teams that appear in the dataset
team_list = list(set(df2013['Home'].unique()) | set(df2013['Away'].unique()))

# Create columns for team levels in each match
df2013['level_team_home'] = 0
df2013['level_team_away'] = 0

# Fill columns with team levels
for team in team_list:
    # Fill with level up to the 19th appearance of the team in the dataset
    team_rows = df2013.loc[((df2013['Home'] == team) | (df2013['Away'] == team))].iloc[:19]
    df2013.loc[((df2013['Home'] == team) & (df2013.index.isin(team_rows.index))), 'level_team_home'] = team_levels_2013[team]
    df2013.loc[((df2013['Away'] == team) & (df2013.index.isin(team_rows.index))), 'level_team_away'] = team_levels_2013[team]
    # Fill with level from the 20th appearance of the team in the dataset
    team_rows = df2013.loc[((df2013['Home'] == team) | (df2013['Away'] == team))].iloc[19:]
    df2013.loc[((df2013['Home'] == team) & (df2013.index.isin(team_rows.index))), 'level_team_home'] = team_levels_2013_rodada19[team]
    df2013.loc[((df2013['Away'] == team) & (df2013.index.isin(team_rows.index))), 'level_team_away'] = team_levels_2013_rodada19[team]

# Define the function to calculate the number of points for the team in the last 5 home/away games
def last_7_points(team, date, location):
    last_7 = df2013[(df2013['Date'] < date) & (((df2013['Home'] == team) & (location == 'home')) | ((df2013['Away'] == team) & (location == 'away')))][-7:]
    points = sum([3 if ((row['Home'] == team and row['Res'] == 'H') or (row['Away'] == team and row['Res'] == 'A')) else (1 if row['Res'] == 'D' else 0) for _, row in last_7.iterrows()])
    return points / 21  # divide by 15 to get the average points per game

# Calculate the number of points for the last 5 home/away games of each team before each game
df2013['last_7_home'] = df2013.apply(lambda row: last_7_points(row['Home'], row['Date'], 'home'), axis=1)
df2013['last_7_away'] = df2013.apply(lambda row: last_7_points(row['Away'], row['Date'], 'away'), axis=1)




<ipython-input-13-88de90f2448c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2013['Date'] = pd.to_datetime(df2013['Date'], format='%d/%m/%Y')


In [ ]:
df2014 = df[df['Season'] == 2014]

# Convert dates to datetime type
df2014['Date'] = pd.to_datetime(df2014['Date'], format='%d/%m/%Y')

# Sort the dataset by date
df2014 = df2014.sort_values('Date')

# Define the function to calculate the number of points for the team in the last 10 rounds
def last_10_points(team, date):
    last_10 = df2014[(df2014['Date'] < date) & ((df2014['Home'] == team) | (df2014['Away'] == team))][-10:]
    points = sum([3 if ((row['Home'] == team and row['Res'] == 'H') or (row['Away'] == team and row['Res'] == 'A')) else (1 if row['Res'] == 'D' else 0) for _, row in last_10.iterrows()])
    return points

# Calculate the number of points for each team in the last 10 rounds before each game
df2014['points_home'] = df2014.apply(lambda row: last_10_points(row['Home'], row['Date']), axis=1)
df2014['points_away'] = df2014.apply(lambda row: last_10_points(row['Away'], row['Date']), axis=1)

df2014['perc_points_home_team'] = df2014['points_home'] / 30
df2014['perc_points_away_team'] = df2014['points_away'] / 30

df2014 = df2014.drop(['points_home', 'points_away'], axis=1)

# Create a list of teams that appear in the dataset
team_list = list(set(df2014['Home'].unique()) | set(df2014['Away'].unique()))

# Create columns for team levels in each match
df2014['level_team_home'] = 0
df2014['level_team_away'] = 0

# Fill columns with team levels
for team in team_list:
    # Fill with level up to the 19th appearance of the team in the dataset
    team_rows = df2014.loc[((df2014['Home'] == team) | (df2014['Away'] == team))].iloc[:19]
    df2014.loc[((df2014['Home'] == team) & (df2014.index.isin(team_rows.index))), 'level_team_home'] = team_levels_2014[team]
    df2014.loc[((df2014['Away'] == team) & (df2014.index.isin(team_rows.index))), 'level_team_away'] = team_levels_2014[team]
    # Fill with level from the 20th appearance of the team in the dataset
    team_rows = df2014.loc[((df2014['Home'] == team) | (df2014['Away'] == team))].iloc[19:]
    df2014.loc[((df2014['Home'] == team) & (df2014.index.isin(team_rows.index))), 'level_team_home'] = team_levels_2014_rodada19[team]
    df2014.loc[((df2014['Away'] == team) & (df2014.index.isin(team_rows.index))), 'level_team_away'] = team_levels_2014_rodada19[team]

# Define the function to calculate the number of points for the team in the last 5 home/away games
def last_7_points(team, date, location):
    last_7 = df2014[(df2014['Date'] < date) & (((df2014['Home'] == team) & (location == 'home')) | ((df2014['Away'] == team) & (location == 'away')))][-7:]
    points = sum([3 if ((row['Home'] == team and row['Res'] == 'H') or (row['Away'] == team and row['Res'] == 'A')) else (1 if row['Res'] == 'D' else 0) for _, row in last_7.iterrows()])
    return points / 21  # divide by 15 to get the average points per game

# Calculate the number of points for the last 5 home/away games of each team before each game
df2014['last_7_home'] = df2014.apply(lambda row: last_7_points(row['Home'], row['Date'], 'home'), axis=1)
df2014['last_7_away'] = df2014.apply(lambda row: last_7_points(row['Away'], row['Date'], 'away'), axis=1)


<ipython-input-14-f004f5c44dc2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2014['Date'] = pd.to_datetime(df2014['Date'], format='%d/%m/%Y')


In [ ]:
df2015 = df[df['Season'] == 2015]

# Convert dates to datetime type
df2015['Date'] = pd.to_datetime(df2015['Date'], format='%d/%m/%Y')

# Sort the dataset by date
df2015 = df2015.sort_values('Date')

# Define the function to calculate the number of points for the team in the last 10 rounds
def last_10_points(team, date):
    last_10 = df2015[(df2015['Date'] < date) & ((df2015['Home'] == team) | (df2015['Away'] == team))][-10:]
    points = sum([3 if ((row['Home'] == team and row['Res'] == 'H') or (row['Away'] == team and row['Res'] == 'A')) else (1 if row['Res'] == 'D' else 0) for _, row in last_10.iterrows()])
    return points

# Calculate the number of points for each team in the last 10 rounds before each game
df2015['points_home'] = df2015.apply(lambda row: last_10_points(row['Home'], row['Date']), axis=1)
df2015['points_away'] = df2015.apply(lambda row: last_10_points(row['Away'], row['Date']), axis=1)

df2015['perc_points_home_team'] = df2015['points_home'] / 30
df2015['perc_points_away_team'] = df2015['points_away'] / 30

df2015 = df2015.drop(['points_home', 'points_away'], axis=1)

# Create a list of teams that appear in the dataset
team_list = list(set(df2015['Home'].unique()) | set(df2015['Away'].unique()))

# Create columns for team levels in each match
df2015['level_team_home'] = 0
df2015['level_team_away'] = 0

# Fill columns with team levels
for team in team_list:
    # Fill with level up to the 19th appearance of the team in the dataset
    team_rows = df2015.loc[((df2015['Home'] == team) | (df2015['Away'] == team))].iloc[:19]
    df2015.loc[((df2015['Home'] == team) & (df2015.index.isin(team_rows.index))), 'level_team_home'] = team_levels_2015[team]
    df2015.loc[((df2015['Away'] == team) & (df2015.index.isin(team_rows.index))), 'level_team_away'] = team_levels_2015[team]
    # Fill with level from the 20th appearance of the team in the dataset
    team_rows = df2015.loc[((df2015['Home'] == team) | (df2015['Away'] == team))].iloc[19:]
    df2015.loc[((df2015['Home'] == team) & (df2015.index.isin(team_rows.index))), 'level_team_home'] = team_levels_2015_rodada19[team]
    df2015.loc[((df2015['Away'] == team) & (df2015.index.isin(team_rows.index))), 'level_team_away'] = team_levels_2015_rodada19[team]

# Define the function to calculate the number of points for the team in the last 5 home/away games
def last_7_points(team, date, location):
    last_7 = df2015[(df2015['Date'] < date) & (((df2015['Home'] == team) & (location == 'home')) | ((df2015['Away'] == team) & (location == 'away')))][-7:]
    points = sum([3 if ((row['Home'] == team and row['Res'] == 'H') or (row['Away'] == team and row['Res'] == 'A')) else (1 if row['Res'] == 'D' else 0) for _, row in last_7.iterrows()])
    return points / 21  # divide by 15 to get the average points per game

# Calculate the number of points for the last 5 home/away games of each team before each game
df2015['last_7_home'] = df2015.apply(lambda row: last_7_points(row['Home'], row['Date'], 'home'), axis=1)
df2015['last_7_away'] = df2015.apply(lambda row: last_7_points(row['Away'], row['Date'], 'away'), axis=1)


<ipython-input-15-1f8f2e2002ab>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2015['Date'] = pd.to_datetime(df2015['Date'], format='%d/%m/%Y')


In [ ]:
df2016 = df[df['Season'] == 2016]

# Convert dates to datetime type
df2016['Date'] = pd.to_datetime(df2016['Date'], format='%d/%m/%Y')

# Sort the dataset by date
df2016 = df2016.sort_values('Date')

# Define the function to calculate the number of points for the team in the last 10 rounds
def last_10_points(team, date):
    last_10 = df2016[(df2016['Date'] < date) & ((df2016['Home'] == team) | (df2016['Away'] == team))][-10:]
    points = sum([3 if ((row['Home'] == team and row['Res'] == 'H') or (row['Away'] == team and row['Res'] == 'A')) else (1 if row['Res'] == 'D' else 0) for _, row in last_10.iterrows()])
    return points

# Calculate the number of points for each team in the last 10 rounds before each game
df2016['points_home'] = df2016.apply(lambda row: last_10_points(row['Home'], row['Date']), axis=1)
df2016['points_away'] = df2016.apply(lambda row: last_10_points(row['Away'], row['Date']), axis=1)

df2016['perc_points_home_team'] = df2016['points_home'] / 30
df2016['perc_points_away_team'] = df2016['points_away'] / 30

df2016 = df2016.drop(['points_home', 'points_away'], axis=1)

# Create a list of teams that appear in the dataset
team_list = list(set(df2016['Home'].unique()) | set(df2016['Away'].unique()))

# Create columns for team levels in each match
df2016['level_team_home'] = 0
df2016['level_team_away'] = 0

# Fill columns with team levels
for team in team_list:
    # Fill with level up to the 19th appearance of the team in the dataset
    team_rows = df2016.loc[((df2016['Home'] == team) | (df2016['Away'] == team))].iloc[:19]
    df2016.loc[((df2016['Home'] == team) & (df2016.index.isin(team_rows.index))), 'level_team_home'] = team_levels_2016[team]
    df2016.loc[((df2016['Away'] == team) & (df2016.index.isin(team_rows.index))), 'level_team_away'] = team_levels_2016[team]
    # Fill with level from the 20th appearance of the team in the dataset
    team_rows = df2016.loc[((df2016['Home'] == team) | (df2016['Away'] == team))].iloc[19:]
    df2016.loc[((df2016['Home'] == team) & (df2016.index.isin(team_rows.index))), 'level_team_home'] = team_levels_2016_rodada19[team]
    df2016.loc[((df2016['Away'] == team) & (df2016.index.isin(team_rows.index))), 'level_team_away'] = team_levels_2016_rodada19[team]

# Define the function to calculate the number of points for the team in the last 5 home/away games
def last_7_points(team, date, location):
    last_7 = df2016[(df2016['Date'] < date) & (((df2016['Home'] == team) & (location == 'home')) | ((df2016['Away'] == team) & (location == 'away')))][-7:]
    points = sum([3 if ((row['Home'] == team and row['Res'] == 'H') or (row['Away'] == team and row['Res'] == 'A')) else (1 if row['Res'] == 'D' else 0) for _, row in last_7.iterrows()])
    return points / 21  # divide by 15 to get the average points per game

# Calculate the number of points for the last 5 home/away games of each team before each game
df2016['last_7_home'] = df2016.apply(lambda row: last_7_points(row['Home'], row['Date'], 'home'), axis=1)
df2016['last_7_away'] = df2016.apply(lambda row: last_7_points(row['Away'], row['Date'], 'away'), axis=1)


<ipython-input-16-d2e1a14db869>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2016['Date'] = pd.to_datetime(df2016['Date'], format='%d/%m/%Y')


In [ ]:
df2013

,Country,League,Season,Date,Time,Home,Away,HG,AG,Res,...,MaxA,AvgH,AvgD,AvgA,perc_points_home_team,perc_points_away_team,level_team_home,level_team_away,last_7_home,last_7_away
380,Brazil,Serie A,2013,2013-05-25,22:30:00,Vasco,Portuguesa,1,0,H,...,5.09,1.86,3.44,4.06,0.000000,0.000000,3.5,3.0,0.000000,0.000000
381,Brazil,Serie A,2013,2013-05-25,22:30:00,Vitoria,Internacional,2,2,D,...,2.77,2.72,3.17,2.52,0.000000,0.000000,2.5,4.0,0.000000,0.000000
382,Brazil,Serie A,2013,2013-05-26,01:00:00,Corinthians,Botafogo RJ,1,1,D,...,5.2,1.82,3.43,4.21,0.000000,0.000000,5.0,4.0,0.000000,0.000000
383,Brazil,Serie A,2013,2013-05-26,20:00:00,Criciuma,Bahia,3,1,H,...,5.25,1.72,3.50,4.70,0.000000,0.000000,2.5,3.0,0.000000,0.000000
384,Brazil,Serie A,2013,2013-05-26,20:00:00,Gremio,Nautico,2,0,H,...,10.5,1.38,4.36,8.19,0.000000,0.000000,4.5,2.5,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754,Brazil,Serie A,2013,2013-12-08,19:00:00,Bahia,Fluminense,1,2,A,...,2.08,3.88,3.36,1.92,0.400000,0.300000,3.0,3.5,0.476190,0.142857
755,Brazil,Serie A,2013,2013-12-08,19:00:00,Botafogo RJ,Criciuma,3,0,H,...,6.05,1.56,3.93,5.53,0.400000,0.566667,4.5,2.5,0.523810,0.476190
756,Brazil,Serie A,2013,2013-12-08,19:00:00,Goias,Santos,0,3,A,...,7.2,1.48,4.12,6.44,0.733333,0.600000,3.5,4.0,0.761905,0.238095
757,Brazil,Serie A,2013,2013-12-08,19:00:00,Internacional,Ponte Preta,0,0,D,...,17.21,1.22,5.80,12.47,0.333333,0.333333,4.0,2.5,0.523810,0.190476


In [ ]:
df2013.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 380 entries, 380 to 759
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Country                380 non-null    object        
 1   League                 380 non-null    object        
 2   Season                 380 non-null    int64         
 3   Date                   380 non-null    datetime64[ns]
 4   Time                   380 non-null    object        
 5   Home                   380 non-null    object        
 6   Away                   380 non-null    object        
 7   HG                     380 non-null    object        
 8   AG                     380 non-null    object        
 9   Res                    380 non-null    object        
 10  PH                     380 non-null    object        
 11  PD                     380 non-null    object        
 12  PA                     380 non-null    object        
 13  Max

In [ ]:
df2012.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 380 entries, 0 to 379
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Country                380 non-null    object        
 1   League                 380 non-null    object        
 2   Season                 380 non-null    int64         
 3   Date                   380 non-null    datetime64[ns]
 4   Time                   380 non-null    object        
 5   Home                   380 non-null    object        
 6   Away                   380 non-null    object        
 7   HG                     380 non-null    object        
 8   AG                     380 non-null    object        
 9   Res                    380 non-null    object        
 10  PH                     380 non-null    object        
 11  PD                     380 non-null    object        
 12  PA                     380 non-null    object        
 13  MaxH 

In [ ]:
df_unificado = pd.concat([df2012, df2013,df2014,df2015,df2016], ignore_index=True)

In [ ]:
df_unificado

,Country,League,Season,Date,Time,Home,Away,HG,AG,Res,...,MaxA,AvgH,AvgD,AvgA,perc_points_home_team,perc_points_away_team,level_team_home,level_team_away,last_7_home,last_7_away
0,Brazil,Serie A,2012,2012-05-19,22:30:00,Palmeiras,Portuguesa,1,1,D,...,5.31,1.69,3.50,4.90,0.000000,0.000000,3.0,2.5,0.000000,0.000000
1,Brazil,Serie A,2012,2012-05-19,22:30:00,Sport Recife,Flamengo RJ,1,1,D,...,2.7,2.59,3.23,2.58,0.000000,0.000000,2.5,3.0,0.000000,0.000000
2,Brazil,Serie A,2012,2012-05-20,01:00:00,Figueirense,Nautico,2,1,H,...,7.22,1.59,3.67,5.64,0.000000,0.000000,3.0,2.5,0.000000,0.000000
3,Brazil,Serie A,2012,2012-05-20,20:00:00,Botafogo RJ,Sao Paulo,4,2,H,...,3.15,2.35,3.26,2.84,0.000000,0.000000,4.0,4.0,0.000000,0.000000
4,Brazil,Serie A,2012,2012-05-20,20:00:00,Corinthians,Fluminense,0,1,A,...,4.41,1.89,3.33,3.89,0.000000,0.000000,5.0,4.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1895,Brazil,Serie A,2016,2016-12-11,19:00:00,Cruzeiro,Corinthians,3,2,H,...,3.07,2.39,3.28,2.87,0.600000,0.466667,3.5,4.0,0.571429,0.285714
1896,Brazil,Serie A,2016,2016-12-11,19:00:00,Chapecoense-SC,Atletico-MG,NaN,NaN,NaN,...,2.68,2.85,3.30,2.67,0.466667,0.433333,3.5,4.5,0.619048,0.285714
1897,Brazil,Serie A,2016,2016-12-11,19:00:00,Atletico-PR,Flamengo RJ,0,0,D,...,3.72,2.13,3.29,3.34,0.466667,0.566667,4.0,4.5,1.000000,0.571429
1898,Brazil,Serie A,2016,2016-12-11,19:00:00,Fluminense,Internacional,1,1,D,...,1.97,3.90,3.47,1.90,0.200000,0.500000,3.5,2.5,0.380952,0.047619


In [ ]:
df_unificado.Res.value_counts()

H    966
D    490
A    443
Name: Res, dtype: int64

In [ ]:
df_unificado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1900 entries, 0 to 1899
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Country                1900 non-null   object        
 1   League                 1900 non-null   object        
 2   Season                 1900 non-null   int64         
 3   Date                   1900 non-null   datetime64[ns]
 4   Time                   1900 non-null   object        
 5   Home                   1900 non-null   object        
 6   Away                   1900 non-null   object        
 7   HG                     1899 non-null   object        
 8   AG                     1899 non-null   object        
 9   Res                    1899 non-null   object        
 10  PH                     1899 non-null   object        
 11  PD                     1899 non-null   object        
 12  PA                     1899 non-null   object        
 13  Max

In [ ]:
# Remover as linhas da primeira até a 50
df_copy = df_unificado.drop(range(0, 50))

# Remover as linhas da 381 até a 430
df_copy = df_unificado.drop(range(381, 431))

In [ ]:
X = df_unificado[['perc_points_home_team','perc_points_away_team','level_team_home','level_team_away', 'last_7_home','last_7_away']]
y = df_unificado[['Res']]

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression

# Separar as features (X) e o target (y)
X = df_unificado[['perc_points_home_team', 'perc_points_away_team', 'level_team_home', 'level_team_away', 'last_7_home', 'last_7_away']]
y = df_unificado[['Res']]

# Codificar as classes com LabelEncoder
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.6, random_state=42)

# Agora você pode treinar o seu modelo de classificação multiclasse
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
model.fit(X_train, y_train)

# Previsões no conjunto de teste
y_pred_prob = model.predict_proba(X_test)

# y_pred_prob é um array contendo as probabilidades de cada classe para cada instância de teste
# Cada linha corresponde a uma instância, e cada coluna corresponde a uma classe

# O resultado probabilístico para cada variável da coluna 'Res' está em y_pred_prob


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
y_pred_prob

array([[2.09183871e-01, 3.03384253e-01, 4.86530654e-01, 9.01221976e-04],
       [2.56563088e-01, 2.43785064e-01, 4.98502280e-01, 1.14956879e-03],
       [1.49891941e-01, 2.12653227e-01, 6.36648215e-01, 8.06616174e-04],
       ...,
       [1.49171076e-01, 2.81689582e-01, 5.68549225e-01, 5.90116762e-04],
       [1.92653683e-01, 2.64412203e-01, 5.41881672e-01, 1.05244172e-03],
       [1.61471906e-01, 2.24737288e-01, 6.12764428e-01, 1.02637788e-03]])

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Previsões no conjunto de teste
y_pred = model.predict(X_test)

# Avaliar métricas de desempenho
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

# Exibir as métricas
print("Accuracy: {:.2f}%".format(accuracy * 100))
print("Precision: {:.2f}%".format(precision * 100))
print("Recall: {:.2f}%".format(recall * 100))
print("F1-score: {:.2f}%".format(f1 * 100))


Accuracy: 50.70%
Precision: 30.21%
Recall: 36.75%
F1-score: 29.91%


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Previsões no conjunto de teste
y_pred = model.predict(X_test)

# Avaliar métricas de desempenho para cada classe
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average=None)
recall = recall_score(y_test, y_pred, average=None)
f1 = f1_score(y_test, y_pred, average=None)

# Exibir as métricas para cada classe
for i in range(len(precision)):
    print("Classe {}: ".format(i))
    print("   Precisão: {:.2f}%".format(precision[i] * 100))
    print("   Recall: {:.2f}%".format(recall[i] * 100))
    print("   F1-score: {:.2f}%".format(f1[i] * 100))
    print()

# Exibir a acurácia global
print("Acurácia: {:.2f}%".format(accuracy * 100))

Classe 0: 
   Precisão: 38.60%
   Recall: 16.06%
   F1-score: 22.68%

Classe 1: 
   Precisão: 0.00%
   Recall: 0.00%
   F1-score: 0.00%

Classe 2: 
   Precisão: 52.05%
   Recall: 94.18%
   F1-score: 67.04%

Acurácia: 50.70%


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Separar as features (X) e o target (y)
X = df_unificado[['perc_points_home_team', 'perc_points_away_team', 'level_team_home', 'level_team_away', 'last_7_home', 'last_7_away']]
y = df_unificado['Res']

# Codificar as classes com LabelEncoder
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

# Criar o modelo de rede neural
model = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)

# Treinar o modelo
model.fit(X_train, y_train)

# Previsões no conjunto de teste
y_pred = model.predict(X_test)

# Avaliar métricas de desempenho
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

# Exibir as métricas
print("Accuracy: {:.2f}%".format(accuracy * 100))
print("Precision: {:.2f}%".format(precision * 100))
print("Recall: {:.2f}%".format(recall * 100))
print("F1-score: {:.2f}%".format(f1 * 100))


Accuracy: 49.65%
Precision: 29.12%
Recall: 37.01%
F1-score: 30.41%


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Previsões no conjunto de teste
y_pred_proba = model.predict_proba(X_test)

# Extrair as probabilidades previstas para cada classe
proba_home = y_pred_proba[:, 0]
proba_draw = y_pred_proba[:, 1]
proba_away = y_pred_proba[:, 2]

# Calcular as odds
odds_home = 1 / proba_home
odds_draw = 1 / proba_draw
odds_away = 1 / proba_away

# Adicionar as odds ao dataframe original
X_test['Odds_Home'] = odds_home
X_test['Odds_Draw'] = odds_draw
X_test['Odds_Away'] = odds_away


In [ ]:
X_test

,perc_points_home_team,perc_points_away_team,level_team_home,level_team_away,last_7_home,last_7_away,Odds_Home,Odds_Draw,Odds_Away
1222,0.100000,0.233333,3.0,3.0,0.095238,0.142857,4.310140,3.751319,2.002113
111,0.500000,0.366667,2.5,2.5,0.476190,0.190476,4.664511,3.912464,1.899072
554,0.566667,0.333333,4.0,3.0,0.714286,0.285714,7.047643,4.846959,1.535133
526,0.466667,0.566667,3.0,4.5,0.333333,0.380952,2.833579,3.469417,2.793585
1310,0.300000,0.633333,3.0,4.0,0.333333,0.476190,3.085712,3.651027,2.494299
...,...,...,...,...,...,...,...,...,...
1318,0.833333,0.633333,4.5,4.5,0.761905,0.333333,5.110984,4.099989,1.784697
1030,0.233333,0.500000,3.5,4.0,0.428571,0.333333,3.661748,3.679292,2.200273
1351,0.500000,0.133333,3.5,3.0,0.619048,0.142857,5.437982,4.308654,1.714783
1242,0.333333,0.400000,3.0,2.0,0.190476,0.047619,7.603534,4.235416,1.586630


#Outra abordagem

In [ ]:
df_unificado.head()

,Country,League,Season,Date,Time,Home,Away,HG,AG,Res,...,MaxA,AvgH,AvgD,AvgA,perc_points_home_team,perc_points_away_team,level_team_home,level_team_away,last_7_home,last_7_away
0,Brazil,Serie A,2012,2012-05-19,22:30:00,Palmeiras,Portuguesa,1,1,D,...,5.31,1.69,3.50,4.90,0.0,0.0,3.0,2.5,0.0,0.0
1,Brazil,Serie A,2012,2012-05-19,22:30:00,Sport Recife,Flamengo RJ,1,1,D,...,2.7,2.59,3.23,2.58,0.0,0.0,2.5,3.0,0.0,0.0
2,Brazil,Serie A,2012,2012-05-20,01:00:00,Figueirense,Nautico,2,1,H,...,7.22,1.59,3.67,5.64,0.0,0.0,3.0,2.5,0.0,0.0
3,Brazil,Serie A,2012,2012-05-20,20:00:00,Botafogo RJ,Sao Paulo,4,2,H,...,3.15,2.35,3.26,2.84,0.0,0.0,4.0,4.0,0.0,0.0
4,Brazil,Serie A,2012,2012-05-20,20:00:00,Corinthians,Fluminense,0,1,A,...,4.41,1.89,3.33,3.89,0.0,0.0,5.0,4.0,0.0,0.0


In [ ]:
df_unificado['perc_points_diff'] =df_unificado['perc_points_home_team']-df_unificado['perc_points_away_team']
df_unificado['level_team_diff'] =df_unificado['level_team_home']-df_unificado['level_team_away']
df_unificado['last_7_diff'] =df_unificado['last_7_home']-df_unificado['last_7_away']


In [ ]:
df_copy = df_unificado[['Home',	'Away','Res','AvgH','AvgD','AvgA','perc_points_diff','level_team_diff','last_7_diff']]

In [ ]:
X = df_unificado[['perc_points_diff','level_team_diff','last_7_diff']]

In [ ]:
y = df_unificado[['Res']]

In [ ]:
# Codificar as classes com LabelEncoder
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# Criar o modelo de rede neural
model = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)

# Treinar o modelo
model.fit(X, y_encoded)

# Previsões no conjunto de teste
y_pred = model.predict(X)

In [ ]:
# Previsões no conjunto de teste
y_pred_proba = model.predict_proba(X)

# Extrair as probabilidades previstas para cada classe
proba_home = y_pred_proba[:, 0]
proba_draw = y_pred_proba[:, 1]
proba_away = y_pred_proba[:, 2]

# Calcular as odds
odds_home = 1 / proba_away
odds_draw = 1 / proba_draw
odds_away = 1 / proba_home

# Adicionar as odds ao dataframe original
df_copy['Odds_Home'] = odds_home
df_copy['Odds_Draw'] = odds_draw
df_copy['Odds_Away'] = odds_away

<ipython-input-40-204b0da237e7>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy['Odds_Home'] = odds_home
<ipython-input-40-204b0da237e7>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy['Odds_Draw'] = odds_draw
<ipython-input-40-204b0da237e7>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

In [ ]:
df_copy.tail(50)

,Home,Away,Res,AvgH,AvgD,AvgA,perc_points_diff,level_team_diff,last_7_diff,Odds_Home,Odds_Draw,Odds_Away
1850,Flamengo RJ,Botafogo RJ,D,1.67,3.65,5.13,-0.100000,0.5,0.190476,1.802391,3.687048,5.762383
1851,Sao Paulo,Corinthians,H,2.09,3.18,3.62,0.133333,-0.5,0.285714,2.279854,3.701870,3.451457
1852,Coritiba,Atletico-MG,H,2.45,3.19,2.87,-0.266667,-1.5,0.428571,3.291006,3.623018,2.382858
1853,Chapecoense-SC,Figueirense,H,1.81,3.34,4.58,0.200000,1.0,0.285714,1.657053,4.264488,6.178067
1854,Santa Cruz,America MG,H,1.80,3.49,4.38,-0.333333,-0.5,0.095238,2.403331,3.470593,3.390456
1855,Vitoria,Atletico-PR,H,1.79,3.39,4.63,-0.266667,-1.0,0.333333,2.969216,3.864715,2.477394
1856,Cruzeiro,Fluminense,H,1.81,3.41,4.45,-0.066667,0.0,0.047619,1.984986,3.545712,4.690094
1857,Ponte Preta,Santos,A,2.34,3.20,3.00,-0.466667,-1.0,0.333333,3.065587,3.800734,2.438061
1858,Palmeiras,Internacional,H,1.59,3.66,5.98,0.333333,2.5,0.619048,1.280188,6.242105,17.047295
1859,Gremio,Sport Recife,A,1.77,3.41,4.75,0.000000,0.5,0.523810,1.701958,4.096951,5.951685


In [ ]:
df_copy[df_copy['Odds_Away']>10].Res.value_counts()

H    94
D    29
A     9
Name: Res, dtype: int64

In [ ]:
df_copy.Res.value_counts()

H    966
D    490
A    443
Name: Res, dtype: int64

In [ ]:
df_copy[df_copy['Odds_Home']>5]

,Home,Away,Res,AvgH,AvgD,AvgA,perc_points_diff,level_team_diff,last_7_diff,Odds_Home,Odds_Draw,Odds_Away
156,Coritiba,Corinthians,A,2.53,3.18,2.67,-0.266667,-3.0,0.190476,6.304375,3.662628,1.759707
636,Nautico,Cruzeiro,A,5.34,3.71,1.61,-0.633333,-3.5,-0.571429,17.676548,18.223074,1.125466
646,Nautico,Botafogo RJ,A,3.15,3.32,2.17,-0.166667,-3.0,-0.333333,6.623660,5.773296,1.479968
751,Nautico,Corinthians,H,3.97,3.31,1.92,-0.466667,-3.0,-0.142857,6.949253,6.096907,1.445072
891,Criciuma,Cruzeiro,D,4.43,3.51,1.77,-0.333333,-3.0,0.000000,6.142151,4.257930,1.660425
1416,Figueirense,Corinthians,A,3.12,3.13,2.31,-0.500000,-3.0,-0.190476,7.333816,6.641329,1.402525
1798,Santa Cruz,Palmeiras,A,3.01,3.19,2.35,-0.533333,-3.5,-0.333333,13.451571,12.681121,1.180962
1807,America MG,Palmeiras,A,7.37,4.24,1.43,-0.433333,-3.0,-0.190476,7.035449,6.272274,1.431946
1830,Santa Cruz,Botafogo RJ,A,2.76,3.20,2.52,-0.666667,-2.5,-0.380952,6.536424,5.573453,1.498244
1878,Santa Cruz,Atletico-MG,D,2.28,3.48,2.90,-0.400000,-3.0,0.047619,6.233947,4.327799,1.643510


In [ ]:
df_copy[df_copy['AvgH']>5]

,Home,Away,Res,AvgH,AvgD,AvgA,perc_points_diff,level_team_diff,last_7_diff,Odds_Home,Odds_Draw,Odds_Away
636,Nautico,Cruzeiro,A,5.34,3.71,1.61,-0.633333,-3.5,-0.571429,17.676548,18.223074,1.125466
792,Figueirense,Santos,A,5.07,3.65,1.65,-0.100000,-1.5,-0.047619,2.958203,3.454518,2.690007
958,Botafogo RJ,Sao Paulo,A,5.13,3.69,1.65,-0.233333,-1.0,0.285714,2.922488,3.858193,2.514129
1628,America MG,Atletico-MG,A,5.98,3.99,1.54,-0.166667,-2.5,0.190476,4.690217,3.119890,2.145497
1807,America MG,Palmeiras,A,7.37,4.24,1.43,-0.433333,-3.0,-0.190476,7.035449,6.272274,1.431946
1864,America MG,Flamengo RJ,A,6.78,4.12,1.47,-0.133333,-2.5,0.000000,4.538114,3.299092,2.099424


In [ ]:
validacao = pd.read_excel('/content/BetsValidacao.xlsx')

In [ ]:
validacao

,Home,Away,perc_points_diff,level_team_diff_,last_7_diff
0,Cruzeiro,Fortaleza,0.0000,-1,0.07
1,Sao Paulo,Atletico-PR,-0.0330,0,0.46
2,Santos,Corinthians,0.1333,0,0.46


In [ ]:
validacao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Home              3 non-null      object 
 1   Away              3 non-null      object 
 2   perc_points_diff  3 non-null      float64
 3   level_team_diff_  3 non-null      int64  
 4   last_7_diff       3 non-null      float64
dtypes: float64(2), int64(1), object(2)
memory usage: 248.0+ bytes


In [ ]:
validacao['level_team_diff']= validacao['level_team_diff_']

In [ ]:
validacao1 = validacao[['perc_points_diff','level_team_diff','last_7_diff']]

In [ ]:
validacao

,Home,Away,perc_points_diff,level_team_diff_,last_7_diff,level_team_diff
0,Cruzeiro,Fortaleza,0.0000,-1,0.07,-1
1,Sao Paulo,Atletico-PR,-0.0330,0,0.46,0
2,Santos,Corinthians,0.1333,0,0.46,0


In [ ]:
y_pred_proba = model.predict_proba(validacao1)

# Extrair as probabilidades previstas para cada classe
proba_home = y_pred_proba[:, 0]
proba_draw = y_pred_proba[:, 1]
proba_away = y_pred_proba[:, 2]

# Calcular as odds
odds_home = 1 / proba_away
odds_draw = 1 / proba_draw
odds_away = 1 / proba_home

# Adicionar as odds ao dataframe original
validacao['Odds_Home'] = odds_home
validacao['Odds_Draw'] = odds_draw
validacao['Odds_Away'] = odds_away

In [ ]:
validacao

,Home,Away,perc_points_diff,level_team_diff_,last_7_diff,level_team_diff,Odds_Home,Odds_Draw,Odds_Away
0,Cruzeiro,Fortaleza,0.0000,-1,0.07,-1,2.533790,3.581809,3.078672
1,Sao Paulo,Atletico-PR,-0.0330,0,0.46,0,1.944671,3.796269,4.511880
2,Santos,Corinthians,0.1333,0,0.46,0,1.944950,3.856061,4.432069


In [ ]:
validacao[['Home', 'Away','Odds_Home','Odds_Draw','Odds_Away']]

,Home,Away,Odds_Home,Odds_Draw,Odds_Away
0,Cruzeiro,Fortaleza,2.533790,3.581809,3.078672
1,Sao Paulo,Atletico-PR,1.944671,3.796269,4.511880
2,Santos,Corinthians,1.944950,3.856061,4.432069


In [ ]:
# Campeonato Brasileiro 2018
df2018 = df[df['Season'] == 2018]
df2018['Date'] = pd.to_datetime(df2018['Date'], format='%d/%m/%Y')
df2018 = df2018.sort_values('Date')

def last_10_points(team, date):
    last_10 = df2018[(df2018['Date'] < date) & ((df2018['Home'] == team) | (df2018['Away'] == team))][-10:]
    points = sum([3 if ((row['Home'] == team and row['Res'] == 'H') or (row['Away'] == team and row['Res'] == 'A')) else (1 if row['Res'] == 'D' else 0) for _, row in last_10.iterrows()])
    return points

df2018['points_home'] = df2018.apply(lambda row: last_10_points(row['Home'], row['Date']), axis=1)
df2018['points_away'] = df2018.apply(lambda row: last_10_points(row['Away'], row['Date']), axis=1)

df2018['perc_points_home_team'] = df2018['points_home'] / 30
df2018['perc_points_away_team'] = df2018['points_away'] / 30

df2018 = df2018.drop(['points_home', 'points_away'], axis=1)

team_list = list(set(df2018['Home'].unique()) | set(df2018['Away'].unique()))

df2018['level_team_home'] = 0
df2018['level_team_away'] = 0

for team in team_list:
    team_rows = df2018.loc[((df2018['Home'] == team) | (df2018['Away'] == team))].iloc[:19]
    df2018.loc[((df2018['Home'] == team) & (df2018.index.isin(team_rows.index))), 'level_team_home'] = team_levels_2018[team]
    df2018.loc[((df2018['Away'] == team) & (df2018.index.isin(team_rows.index))), 'level_team_away'] = team_levels_2018[team]
    team_rows = df2018.loc[((df2018['Home'] == team) | (df2018['Away'] == team))].iloc[19:]
    df2018.loc[((df2018['Home'] == team) & (df2018.index.isin(team_rows.index))), 'level_team_home'] = team_levels_2018_rodada19[team]
    df2018.loc[((df2018['Away'] == team) & (df2018.index.isin(team_rows.index))), 'level_team_away'] = team_levels_2018_rodada19[team]

def last_7_points(team, date, location):
    last_7 = df2018[(df2018['Date'] < date) & (((df2018['Home'] == team) & (location == 'home')) | ((df2018['Away'] == team) & (location == 'away')))][-7:]
    points = sum([3 if ((row['Home'] == team and row['Res'] == 'H') or (row['Away'] == team and row['Res'] == 'A')) else (1 if row['Res'] == 'D' else 0) for _, row in last_7.iterrows()])
    return points / 21

df2018['last_7_home'] = df2018.apply(lambda row: last_7_points(row['Home'], row['Date'], 'home'), axis=1)
df2018['last_7_away'] = df2018.apply(lambda row: last_7_points(row['Away'], row['Date'], 'away'), axis=1)


# Campeonato Brasileiro 2019
df2019 = df[df['Season'] == 2019]
df2019['Date'] = pd.to_datetime(df2019['Date'], format='%d/%m/%Y')
df2019 = df2019.sort_values('Date')

def last_10_points(team, date):
    last_10 = df2019[(df2019['Date'] < date) & ((df2019['Home'] == team) | (df2019['Away'] == team))][-10:]
    points = sum([3 if ((row['Home'] == team and row['Res'] == 'H') or (row['Away'] == team and row['Res'] == 'A')) else (1 if row['Res'] == 'D' else 0) for _, row in last_10.iterrows()])
    return points

df2019['points_home'] = df2019.apply(lambda row: last_10_points(row['Home'], row['Date']), axis=1)
df2019['points_away'] = df2019.apply(lambda row: last_10_points(row['Away'], row['Date']), axis=1)

df2019['perc_points_home_team'] = df2019['points_home'] / 30
df2019['perc_points_away_team'] = df2019['points_away'] / 30

df2019 = df2019.drop(['points_home', 'points_away'], axis=1)

team_list = list(set(df2019['Home'].unique()) | set(df2019['Away'].unique()))

df2019['level_team_home'] = 0
df2019['level_team_away'] = 0

for team in team_list:
    team_rows = df2019.loc[((df2019['Home'] == team) | (df2019['Away'] == team))].iloc[:19]
    df2019.loc[((df2019['Home'] == team) & (df2019.index.isin(team_rows.index))), 'level_team_home'] = team_levels_2019[team]
    df2019.loc[((df2019['Away'] == team) & (df2019.index.isin(team_rows.index))), 'level_team_away'] = team_levels_2019[team]
    team_rows = df2019.loc[((df2019['Home'] == team) | (df2019['Away'] == team))].iloc[19:]
    df2019.loc[((df2019['Home'] == team) & (df2019.index.isin(team_rows.index))), 'level_team_home'] = team_levels_2019_rodada19[team]
    df2019.loc[((df2019['Away'] == team) & (df2019.index.isin(team_rows.index))), 'level_team_away'] = team_levels_2019_rodada19[team]

def last_7_points(team, date, location):
    last_7 = df2019[(df2019['Date'] < date) & (((df2019['Home'] == team) & (location == 'home')) | ((df2019['Away'] == team) & (location == 'away')))][-7:]
    points = sum([3 if ((row['Home'] == team and row['Res'] == 'H') or (row['Away'] == team and row['Res'] == 'A')) else (1 if row['Res'] == 'D' else 0) for _, row in last_7.iterrows()])
    return points / 21

df2019['last_7_home'] = df2019.apply(lambda row: last_7_points(row['Home'], row['Date'], 'home'), axis=1)
df2019['last_7_away'] = df2019.apply(lambda row: last_7_points(row['Away'], row['Date'], 'away'), axis=1)


<ipython-input-55-ca53c2c29c82>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2018['Date'] = pd.to_datetime(df2018['Date'], format='%d/%m/%Y')
<ipython-input-55-ca53c2c29c82>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2019['Date'] = pd.to_datetime(df2019['Date'], format='%d/%m/%Y')


In [ ]:
df_test = pd.concat([df2018,df2019], ignore_index=True)

In [ ]:
df_test

,Country,League,Season,Date,Time,Home,Away,HG,AG,Res,...,MaxA,AvgH,AvgD,AvgA,perc_points_home_team,perc_points_away_team,level_team_home,level_team_away,last_7_home,last_7_away
0,Brazil,Serie A,2018,2018-04-14,20:00:00,Cruzeiro,Gremio,0,1,A,...,4.87,1.98,3.16,4.15,0.000000,0.000000,4.0,4.5,0.000000,0.000000
1,Brazil,Serie A,2018,2018-04-14,23:00:00,Vitoria,Flamengo RJ,2,2,D,...,2.24,3.40,3.31,2.14,0.000000,0.000000,2.5,5.0,0.000000,0.000000
2,Brazil,Serie A,2018,2018-04-15,01:00:00,Santos,Ceara,2,0,H,...,7.1,1.55,3.88,6.12,0.000000,0.000000,4.0,3.0,0.000000,0.000000
3,Brazil,Serie A,2018,2018-04-15,15:00:00,America MG,Sport Recife,3,0,H,...,4.51,1.96,3.27,4.01,0.000000,0.000000,2.0,3.0,0.000000,0.000000
4,Brazil,Serie A,2018,2018-04-15,20:00:00,Corinthians,Fluminense,2,1,H,...,7,1.62,3.55,6.03,0.000000,0.000000,3.5,3.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,Brazil,Serie A,2019,2019-12-08,19:00:00,CSA,Sao Paulo,1,2,A,...,2.36,3.43,3.21,2.17,0.200000,0.466667,2.5,4.0,0.476190,0.285714
756,Brazil,Serie A,2019,2019-12-08,19:00:00,Fortaleza,Bahia,2,1,H,...,3.9,2.02,3.51,3.54,0.633333,0.266667,3.0,3.5,0.761905,0.333333
757,Brazil,Serie A,2019,2019-12-08,19:00:00,Goias,Gremio,3,2,H,...,3.87,2.01,3.52,3.53,0.366667,0.800000,3.5,4.5,0.523810,0.571429
758,Brazil,Serie A,2019,2019-12-08,19:00:00,Internacional,Atletico-MG,2,1,H,...,6,1.57,4.09,5.42,0.400000,0.433333,4.0,4.5,0.333333,0.238095


In [ ]:
df_test['perc_points_diff'] =df_test['perc_points_home_team']-df_test['perc_points_away_team']
df_test['level_team_diff'] =df_test['level_team_home']-df_test['level_team_away']
df_test['last_7_diff'] =df_test['last_7_home']-df_test['last_7_away']

In [ ]:
validacao2 = df_test[['perc_points_diff','level_team_diff','last_7_diff']]

In [ ]:
y_pred_proba = model.predict_proba(validacao2)

# Extrair as probabilidades previstas para cada classe
proba_home = y_pred_proba[:, 0]
proba_draw = y_pred_proba[:, 1]
proba_away = y_pred_proba[:, 2]

# Calcular as odds
odds_home = 1 / proba_away
odds_draw = 1 / proba_draw
odds_away = 1 / proba_home

# Adicionar as odds ao dataframe original
df_test['Odds_Home'] = odds_home
df_test['Odds_Draw'] = odds_draw
df_test['Odds_Away'] = odds_away

In [ ]:
comparacao = df_test[['Home',	'Away','Res','AvgH','AvgD','AvgA','Odds_Home', 'Odds_Draw','Odds_Away']]

In [ ]:
comparacao.iloc[120:161]

,Home,Away,Res,AvgH,AvgD,AvgA,Odds_Home,Odds_Draw,Odds_Away
120,Vitoria,Parana,H,1.88,3.33,4.30,1.952961,3.730286,4.567349
121,Corinthians,Botafogo RJ,H,1.92,3.28,4.18,1.946781,3.684383,4.667826
122,Flamengo RJ,Sao Paulo,A,2.01,3.25,3.85,1.820065,3.839988,5.271328
123,Gremio,Atletico-MG,H,1.60,3.70,5.91,1.958667,3.565598,4.804870
124,Chapecoense-SC,Bahia,D,2.02,3.03,4.16,2.372031,3.549468,3.384697
125,Cruzeiro,America MG,H,1.48,4.00,7.32,1.374278,5.369663,11.613722
126,Santos,Palmeiras,D,2.57,3.13,2.80,2.711788,3.689607,2.784207
127,Vasco,Fluminense,D,2.25,3.18,3.25,1.941253,3.782299,4.553211
128,Atletico-PR,Internacional,D,2.61,2.95,2.89,2.392973,3.223817,3.686207
129,Flamengo RJ,Botafogo RJ,H,1.67,3.51,5.49,1.473737,4.802950,8.832889


In [ ]:
df_test[df_test['AvgH']<1.5].Res.value_counts()

H    97
D    25
A    12
Name: Res, dtype: int64

In [ ]:
df_test[df_test['Odds_Home']<1.5].Res.value_counts()

H    90
D    15
A     8
Name: Res, dtype: int64

In [ ]:
comparacao[comparacao['Home']=='Parana']

,Home,Away,Res,AvgH,AvgD,AvgA,Odds_Home,Odds_Draw,Odds_Away
12,Parana,Corinthians,A,3.79,2.96,2.16,2.597498,3.607761,2.970149
21,Parana,Sport Recife,A,2.21,3.17,3.35,2.183364,3.578187,3.830466
57,Parana,Gremio,D,4.65,3.32,1.82,3.602147,3.311665,2.380569
66,Parana,Atletico-PR,D,2.56,3.02,2.91,2.958203,3.454518,2.690007
88,Parana,Fluminense,H,2.51,3.05,2.94,2.340640,3.291503,3.728988
97,Parana,Bahia,H,2.47,2.93,3.13,2.724596,3.737888,2.745359
112,Parana,Cruzeiro,D,3.74,2.96,2.17,3.154908,3.598955,2.471641
133,Parana,America MG,H,2.29,2.99,3.40,1.777605,3.904729,5.526712
165,Parana,Ceara,A,2.07,3.03,3.97,2.357604,3.635856,3.338359
174,Parana,Botafogo RJ,D,2.69,2.94,2.84,2.780062,3.713875,2.703182


In [ ]:
comparacao[comparacao['Away']=='Parana']

,Home,Away,Res,AvgH,AvgD,AvgA,Odds_Home,Odds_Draw,Odds_Away
9,Sao Paulo,Parana,H,1.42,4.19,8.20,1.395523,5.488868,9.878914
38,Chapecoense-SC,Parana,D,1.74,3.34,5.27,1.916279,3.538025,5.128677
43,Santos,Parana,H,1.53,3.83,6.69,1.515010,4.758743,7.706765
70,Vasco,Parana,H,1.94,3.14,4.29,1.871572,3.731174,5.071519
103,Flamengo RJ,Parana,H,1.29,4.77,12.30,1.295543,6.159820,15.202582
120,Vitoria,Parana,H,1.88,3.33,4.30,1.952961,3.730286,4.567349
144,Atletico-MG,Parana,H,1.47,4.17,6.94,1.329289,5.620232,14.330045
149,Palmeiras,Parana,H,1.31,4.85,10.39,1.260904,6.477029,19.038646
184,Internacional,Parana,H,1.41,4.22,8.32,1.343721,5.534411,13.314965
197,Corinthians,Parana,H,1.69,3.29,6.02,1.585215,4.397558,7.059845


In [ ]:
comparacao[comparacao['AvgH']> comparacao['Odds_Home']]

,Home,Away,Res,AvgH,AvgD,AvgA,Odds_Home,Odds_Draw,Odds_Away
5,Internacional,Bahia,H,1.74,3.42,5.05,1.694127,4.006874,6.250424
7,Atletico-PR,Chapecoense-SC,H,2.08,3.17,3.72,1.694127,4.006874,6.250424
8,Botafogo RJ,Palmeiras,D,3.61,3.23,2.08,2.831758,3.374255,2.859597
9,Sao Paulo,Parana,H,1.42,4.19,8.20,1.395523,5.488868,9.878914
10,Flamengo RJ,America MG,H,1.41,4.43,8.00,1.222679,7.667199,19.343305
...,...,...,...,...,...,...,...,...,...
749,Sao Paulo,Internacional,H,2.13,2.95,3.89,1.980941,3.696326,4.471310
750,Santos,Flamengo RJ,H,2.65,3.54,2.47,2.286670,3.497958,3.627090
752,Botafogo RJ,Ceara,D,3.10,2.51,2.90,1.944427,3.789992,4.523310
753,Corinthians,Fluminense,A,2.11,3.19,3.60,1.633202,4.138445,6.852796


In [ ]:
comparacao[(comparacao['AvgH']> comparacao['Odds_Home']) & (comparacao['AvgH']>1.7) & (comparacao['AvgH']<2)].Res.value_counts()

H    37
D    13
A     7
Name: Res, dtype: int64

In [ ]:
comparacao[(comparacao['AvgH']> comparacao['Odds_Home']) & (comparacao['AvgH']>1.7) & (comparacao['AvgH']<2)]

,Home,Away,Res,AvgH,AvgD,AvgA,Odds_Home,Odds_Draw,Odds_Away
5,Internacional,Bahia,H,1.74,3.42,5.05,1.694127,4.006874,6.250424
24,Atletico-MG,Corinthians,H,1.97,3.28,3.96,1.675056,3.998333,6.546744
48,Ceara,America MG,D,1.95,3.25,4.04,1.657548,4.017502,6.773165
49,Atletico-MG,Cruzeiro,H,1.98,3.29,3.89,1.849259,3.729845,5.244911
63,Internacional,Corinthians,H,1.81,3.28,4.82,1.639358,4.096819,6.860133
68,Botafogo RJ,Vitoria,D,1.84,3.62,4.07,1.650943,4.157088,6.511331
70,Vasco,Parana,H,1.94,3.14,4.29,1.871572,3.731174,5.071519
85,Flamengo RJ,Corinthians,H,1.74,3.42,5.09,1.489750,4.724641,8.543279
108,Corinthians,Vitoria,D,1.80,3.38,4.69,1.633043,4.292513,6.471170
131,Bahia,Vitoria,H,1.84,3.49,4.28,1.578722,4.221112,7.718951


In [ ]:
team_levels_2020 = {
    "Flamengo RJ": 5,
    "Internacional": 4.5,
    "Atletico-MG": 4.5,
    "Sao Paulo": 4.5,
    "Palmeiras": 4.5,
    "Gremio": 4,
    "Fluminense": 4,
    "Santos": 3.5,
    "Corinthians": 3.5,
    "Athletico-PR": 3.5,
    "Bahia": 3,
    "Sport Recife": 2.5,
    "Fortaleza": 3,
    "Ceara": 3,
    "Botafogo RJ": 2.5,
    "Vasco": 2.5,
    "Coritiba": 2.5,
    "Goias": 2,
    "Bragantino": 3,
    "Atletico GO": 2
}

team_levels_2020_rodada19 = {
    "Flamengo RJ": 5,
    "Internacional": 4.5,
    "Atletico-MG": 4.5,
    "Sao Paulo": 4.5,
    "Palmeiras": 4,
    "Gremio": 4,
    "Fluminense": 4,
    "Santos": 3.5,
    "Corinthians": 3.5,
    "Athletico-PR": 3.5,
    "Bahia": 3,
    "Sport Recife": 3,
    "Fortaleza": 3,
    "Ceara": 3,
    "Botafogo RJ": 2.5,
    "Vasco": 2.5,
    "Coritiba": 2.5,
    "Goias": 2,
    "Bragantino": 3.5,
    "Atletico GO": 1.5
}


In [ ]:
team_levels_2021 = {
    "Flamengo RJ": 5,
    "Palmeiras": 4.5,
    "Atletico-MG": 4.5,
    "Fortaleza": 3.5,
    "Corinthians": 4,
    "Fluminense": 4,
    "Athletico-PR": 4,
    "Internacional": 3.5,
    "Ceara": 3.5,
    "Bragantino": 3.5,
    "Santos": 3,
    "Bahia": 3,
    "Sao Paulo": 3.5,
    "Juventude": 2.5,
    "Sport Recife": 2.5,
    "Cuiaba": 2.5,
    "America MG": 2,
    "Gremio": 3.5,
    "Chapecoense-SC": 2,
    "Atletico GO": 2.5
}

team_levels_2021_rodada19 = {
    "Flamengo RJ": 5,
    "Palmeiras": 4.5,
    "Atletico-MG": 4.5,
    "Fortaleza": 4.5,
    "Corinthians": 4,
    "Fluminense": 4,
    "Athletico-PR": 4,
    "Internacional": 3.5,
    "Ceara": 3.5,
    "Bragantino": 4,
    "Santos": 3,
    "Bahia": 3,
    "Sao Paulo": 3,
    "Juventude": 2.5,
    "Sport Recife": 2.5,
    "Cuiaba": 2.5,
    "America MG": 3,
    "Gremio": 2.5,
    "Chapecoense-SC": 1.5,
    "Atletico GO": 3
}


In [ ]:
team_levels_2022 = {
    "Flamengo RJ": 5,
    "Palmeiras": 5,
    "Atletico-MG": 4.5,
    "Fortaleza": 4,
    "Corinthians": 4,
    "Fluminense": 4,
    "Athletico-PR": 4,
    "Internacional": 3.5,
    "Ceara": 3.5,
    "Bragantino": 3.5,
    "Santos": 3,
    "Avai": 2.5,
    "Sao Paulo": 3.5,
    "Juventude": 2.5,
    "Atletico GO": 2.5,
    "Cuiaba": 2.5,
    "America MG": 2.5,
    "Botafogo RJ": 3,
    "Coritiba": 2,
    "Goias": 2.5
}

team_levels_2022_rodada19 = {
    "Flamengo RJ": 4.5,
    "Palmeiras": 5,
    "Atletico-MG": 4,
    "Fortaleza": 4,
    "Corinthians": 4,
    "Fluminense": 4,
    "Athletico-PR": 4,
    "Internacional": 4.5,
    "Ceara": 2.5,
    "Bragantino": 3,
    "Santos": 3,
    "Avai": 2,
    "Sao Paulo": 3.5,
    "Juventude": 1.5,
    "Atletico GO": 2,
    "Cuiaba": 2.5,
    "America MG": 3,
    "Botafogo RJ": 3,
    "Coritiba": 2.5,
    "Goias": 3
}


In [ ]:
import pandas as pd

# Campeonato Brasileiro 2020
df2020 = df[df['Season'] == 2020]
df2020['Date'] = pd.to_datetime(df2020['Date'], format='%d/%m/%Y')
df2020 = df2020.sort_values('Date')

def last_10_points(team, date):
    last_10 = df2020[(df2020['Date'] < date) & ((df2020['Home'] == team) | (df2020['Away'] == team))][-10:]
    points = sum([3 if ((row['Home'] == team and row['Res'] == 'H') or (row['Away'] == team and row['Res'] == 'A')) else (1 if row['Res'] == 'D' else 0) for _, row in last_10.iterrows()])
    return points

df2020['points_home'] = df2020.apply(lambda row: last_10_points(row['Home'], row['Date']), axis=1)
df2020['points_away'] = df2020.apply(lambda row: last_10_points(row['Away'], row['Date']), axis=1)

df2020['perc_points_home_team'] = df2020['points_home'] / 30
df2020['perc_points_away_team'] = df2020['points_away'] / 30

df2020 = df2020.drop(['points_home', 'points_away'], axis=1)

team_list = list(set(df2020['Home'].unique()) | set(df2020['Away'].unique()))

df2020['level_team_home'] = 0
df2020['level_team_away'] = 0

for team in team_list:
    team_rows = df2020.loc[((df2020['Home'] == team) | (df2020['Away'] == team))].iloc[:19]
    df2020.loc[((df2020['Home'] == team) & (df2020.index.isin(team_rows.index))), 'level_team_home'] = team_levels_2020[team]
    df2020.loc[((df2020['Away'] == team) & (df2020.index.isin(team_rows.index))), 'level_team_away'] = team_levels_2020[team]
    team_rows = df2020.loc[((df2020['Home'] == team) | (df2020['Away'] == team))].iloc[19:]
    df2020.loc[((df2020['Home'] == team) & (df2020.index.isin(team_rows.index))), 'level_team_home'] = team_levels_2020_rodada19[team]
    df2020.loc[((df2020['Away'] == team) & (df2020.index.isin(team_rows.index))), 'level_team_away'] = team_levels_2020_rodada19[team]

def last_7_points(team, date, location):
    last_7 = df2020[(df2020['Date'] < date) & (((df2020['Home'] == team) & (location == 'home')) | ((df2020['Away'] == team) & (location == 'away')))][-7:]
    points = sum([3 if ((row['Home'] == team and row['Res'] == 'H') or (row['Away'] == team and row['Res'] == 'A')) else (1 if row['Res'] == 'D' else 0) for _, row in last_7.iterrows()])
    return points / 21

df2020['last_7_home'] = df2020.apply(lambda row: last_7_points(row['Home'], row['Date'], 'home'), axis=1)
df2020['last_7_away'] = df2020.apply(lambda row: last_7_points(row['Away'], row['Date'], 'away'), axis=1)



<ipython-input-73-19e60424faa1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2020['Date'] = pd.to_datetime(df2020['Date'], format='%d/%m/%Y')


In [ ]:
import pandas as pd

# Campeonato Brasileiro 2021
df2021 = df[df['Season'] == 2021]
df2021['Date'] = pd.to_datetime(df2021['Date'], format='%d/%m/%Y')
df2021 = df2021.sort_values('Date')

def last_10_points(team, date):
    last_10 = df2021[(df2021['Date'] < date) & ((df2021['Home'] == team) | (df2021['Away'] == team))][-10:]
    points = sum([3 if ((row['Home'] == team and row['Res'] == 'H') or (row['Away'] == team and row['Res'] == 'A')) else (1 if row['Res'] == 'D' else 0) for _, row in last_10.iterrows()])
    return points

df2021['points_home'] = df2021.apply(lambda row: last_10_points(row['Home'], row['Date']), axis=1)
df2021['points_away'] = df2021.apply(lambda row: last_10_points(row['Away'], row['Date']), axis=1)

df2021['perc_points_home_team'] = df2021['points_home'] / 30
df2021['perc_points_away_team'] = df2021['points_away'] / 30

df2021 = df2021.drop(['points_home', 'points_away'], axis=1)

team_list = list(set(df2021['Home'].unique()) | set(df2021['Away'].unique()))

df2021['level_team_home'] = 0
df2021['level_team_away'] = 0

for team in team_list:
    team_rows = df2021.loc[((df2021['Home'] == team) | (df2021['Away'] == team))].iloc[:19]
    df2021.loc[((df2021['Home'] == team) & (df2021.index.isin(team_rows.index))), 'level_team_home'] = team_levels_2021[team]
    df2021.loc[((df2021['Away'] == team) & (df2021.index.isin(team_rows.index))), 'level_team_away'] = team_levels_2021[team]
    team_rows = df2021.loc[((df2021['Home'] == team) | (df2021['Away'] == team))].iloc[19:]
    df2021.loc[((df2021['Home'] == team) & (df2021.index.isin(team_rows.index))), 'level_team_home'] = team_levels_2021_rodada19[team]
    df2021.loc[((df2021['Away'] == team) & (df2021.index.isin(team_rows.index))), 'level_team_away'] = team_levels_2021_rodada19[team]

def last_7_points(team, date, location):
    last_7 = df2021[(df2021['Date'] < date) & (((df2021['Home'] == team) & (location == 'home')) | ((df2021['Away'] == team) & (location == 'away')))][-7:]
    points = sum([3 if ((row['Home'] == team and row['Res'] == 'H') or (row['Away'] == team and row['Res'] == 'A')) else (1 if row['Res'] == 'D' else 0) for _, row in last_7.iterrows()])
    return points / 21

df2021['last_7_home'] = df2021.apply(lambda row: last_7_points(row['Home'], row['Date'], 'home'), axis=1)
df2021['last_7_away'] = df2021.apply(lambda row: last_7_points(row['Away'], row['Date'], 'away'), axis=1)

# Campeonato Brasileiro 2022
df2022 = df[df['Season'] == 2022]
df2022['Date'] = pd.to_datetime(df2022['Date'], format='%d/%m/%Y')
df2022 = df2022.sort_values('Date')

def last_10_points(team, date):
    last_10 = df2022[(df2022['Date'] < date) & ((df2022['Home'] == team) | (df2022['Away'] == team))][-10:]
    points = sum([3 if ((row['Home'] == team and row['Res'] == 'H') or (row['Away'] == team and row['Res'] == 'A')) else (1 if row['Res'] == 'D' else 0) for _, row in last_10.iterrows()])
    return points

df2022['points_home'] = df2022.apply(lambda row: last_10_points(row['Home'], row['Date']), axis=1)
df2022['points_away'] = df2022.apply(lambda row: last_10_points(row['Away'], row['Date']), axis=1)

df2022['perc_points_home_team'] = df2022['points_home'] / 30
df2022['perc_points_away_team'] = df2022['points_away'] / 30

df2022 = df2022.drop(['points_home', 'points_away'], axis=1)

team_list = list(set(df2022['Home'].unique()) | set(df2022['Away'].unique()))

df2022['level_team_home'] = 0
df2022['level_team_away'] = 0

for team in team_list:
    team_rows = df2022.loc[((df2022['Home'] == team) | (df2022['Away'] == team))].iloc[:19]
    df2022.loc[((df2022['Home'] == team) & (df2022.index.isin(team_rows.index))), 'level_team_home'] = team_levels_2022[team]
    df2022.loc[((df2022['Away'] == team) & (df2022.index.isin(team_rows.index))), 'level_team_away'] = team_levels_2022[team]
    team_rows = df2022.loc[((df2022['Home'] == team) | (df2022['Away'] == team))].iloc[19:]
    df2022.loc[((df2022['Home'] == team) & (df2022.index.isin(team_rows.index))), 'level_team_home'] = team_levels_2022_rodada19[team]
    df2022.loc[((df2022['Away'] == team) & (df2022.index.isin(team_rows.index))), 'level_team_away'] = team_levels_2022_rodada19[team]

def last_7_points(team, date, location):
    last_7 = df2022[(df2022['Date'] < date) & (((df2022['Home'] == team) & (location == 'home')) | ((df2022['Away'] == team) & (location == 'away')))][-7:]
    points = sum([3 if ((row['Home'] == team and row['Res'] == 'H') or (row['Away'] == team and row['Res'] == 'A')) else (1 if row['Res'] == 'D' else 0) for _, row in last_7.iterrows()])
    return points / 21

df2022['last_7_home'] = df2022.apply(lambda row: last_7_points(row['Home'], row['Date'], 'home'), axis=1)
df2022['last_7_away'] = df2022.apply(lambda row: last_7_points(row['Away'], row['Date'], 'away'), axis=1)


<ipython-input-74-4efd06816614>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2021['Date'] = pd.to_datetime(df2021['Date'], format='%d/%m/%Y')
<ipython-input-74-4efd06816614>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2022['Date'] = pd.to_datetime(df2022['Date'], format='%d/%m/%Y')


In [ ]:
validacao_atual = pd.concat([df2020,df2021,df2022], ignore_index=True)

In [ ]:
validacao_atual['perc_points_diff'] =validacao_atual['perc_points_home_team']-validacao_atual['perc_points_away_team']
validacao_atual['level_team_diff'] =validacao_atual['level_team_home']-validacao_atual['level_team_away']
validacao_atual['last_7_diff'] =validacao_atual['last_7_home']-validacao_atual['last_7_away']

In [ ]:
validacao4 = validacao_atual[['perc_points_diff','level_team_diff','last_7_diff']]

In [ ]:
y_pred_proba = model.predict_proba(validacao4)

# Extrair as probabilidades previstas para cada classe
proba_home = y_pred_proba[:, 0]
proba_draw = y_pred_proba[:, 1]
proba_away = y_pred_proba[:, 2]

# Calcular as odds
odds_home = 1 / proba_away
odds_draw = 1 / proba_draw
odds_away = 1 / proba_home

# Adicionar as odds ao dataframe original
validacao_atual['Odds_Home'] = odds_home
validacao_atual['Odds_Draw'] = odds_draw
validacao_atual['Odds_Away'] = odds_away

In [ ]:
comparacao2 = validacao_atual[['Home',	'Away','Res','AvgH','AvgD','AvgA','Odds_Home', 'Odds_Draw','Odds_Away']]
comparacao2

,Home,Away,Res,AvgH,AvgD,AvgA,Odds_Home,Odds_Draw,Odds_Away
0,Fortaleza,Athletico-PR,A,2.02,3.25,3.77,2.162352,3.580250,3.895542
1,Coritiba,Internacional,A,3.77,3.06,2.11,3.404066,3.038466,2.654742
2,Sport Recife,Ceara,H,2.91,2.93,2.61,2.162352,3.580250,3.895542
3,Flamengo RJ,Atletico-MG,A,1.47,4.46,6.27,1.909678,3.495467,5.270161
4,Santos,Bragantino,D,2.15,3.25,3.39,1.909678,3.495467,5.270161
...,...,...,...,...,...,...,...,...,...
1135,Bragantino,Fluminense,A,2.40,3.53,2.76,2.640902,3.439807,3.032175
1136,Athletico-PR,Botafogo RJ,H,2.18,3.26,3.38,1.723851,3.565641,7.178185
1137,America MG,Atletico GO,D,1.59,4.21,5.14,1.633505,4.188141,6.715755
1138,Ceara,Juventude,H,1.67,3.84,4.98,1.621259,4.226726,6.827622


In [ ]:
comparacao2[(comparacao2['AvgH']> comparacao2['Odds_Home']) & (comparacao2['AvgH']>1.7) & (comparacao2['AvgH']<2)].head(34)

,Home,Away,Res,AvgH,AvgD,AvgA,Odds_Home,Odds_Draw,Odds_Away
7,Atletico-MG,Corinthians,H,1.98,3.21,4.00,1.712619,4.027408,5.965763
9,Internacional,Santos,H,1.85,3.21,4.67,1.706381,4.020153,6.058977
29,Corinthians,Coritiba,H,1.83,3.14,5.00,1.700247,4.013105,6.153945
33,Internacional,Atletico GO,H,1.75,3.36,5.06,1.294317,6.364461,14.231389
35,Sao Paulo,Bahia,D,1.74,3.41,5.13,1.479777,4.785428,8.679289
42,Palmeiras,Santos,H,1.96,3.11,4.27,1.701211,3.942997,6.312757
45,Sao Paulo,Athletico-PR,H,1.97,3.15,4.13,1.687184,4.044120,6.255544
55,Fluminense,Atletico GO,D,1.73,3.38,5.19,1.382053,5.374184,11.067455
66,Corinthians,Botafogo RJ,D,1.79,3.37,4.79,1.687184,4.044120,6.255544
69,Sao Paulo,Fluminense,H,1.88,3.23,4.49,1.797008,3.845768,5.462416


In [ ]:
comparacao2[(comparacao2['AvgH']> comparacao2['Odds_Home']) & (comparacao2['AvgH']>1.7) & (comparacao2['AvgH']<2)].head(30).Res.value_counts()

H    21
D     6
A     3
Name: Res, dtype: int64

In [ ]:
coeficientes = model.coefs_

# Calcule a importância relativa de cada variável
importancias = [abs(coef).mean() for coef in coeficientes]

# Imprima a importância de cada variável
for i, feature in enumerate(X.columns):
    print(f'{feature}: {importancias[i]}')

perc_points_diff: 0.1473950889769033
level_team_diff: 0.08444363045564092
last_7_diff: 0.17297683342529224


In [ ]:
df_test

,Country,League,Season,Date,Time,Home,Away,HG,AG,Res,...,level_team_home,level_team_away,last_7_home,last_7_away,perc_points_diff,level_team_diff,last_7_diff,Odds_Home,Odds_Draw,Odds_Away
0,Brazil,Serie A,2018,2018-04-14,20:00:00,Cruzeiro,Gremio,0,1,A,...,4.0,4.5,0.000000,0.000000,0.000000,-0.5,0.000000,2.162352,3.580250,3.895542
1,Brazil,Serie A,2018,2018-04-14,23:00:00,Vitoria,Flamengo RJ,2,2,D,...,2.5,5.0,0.000000,0.000000,0.000000,-2.5,0.000000,4.404367,2.924321,2.321494
2,Brazil,Serie A,2018,2018-04-15,01:00:00,Santos,Ceara,2,0,H,...,4.0,3.0,0.000000,0.000000,0.000000,1.0,0.000000,1.694127,4.006874,6.250424
3,Brazil,Serie A,2018,2018-04-15,15:00:00,America MG,Sport Recife,3,0,H,...,2.0,3.0,0.000000,0.000000,0.000000,-1.0,0.000000,2.481997,3.535704,3.195530
4,Brazil,Serie A,2018,2018-04-15,20:00:00,Corinthians,Fluminense,2,1,H,...,3.5,3.0,0.000000,0.000000,0.000000,0.5,0.000000,1.909678,3.495467,5.270161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,Brazil,Serie A,2019,2019-12-08,19:00:00,CSA,Sao Paulo,1,2,A,...,2.5,4.0,0.476190,0.285714,-0.266667,-1.5,0.190476,3.178737,3.727192,2.400569
756,Brazil,Serie A,2019,2019-12-08,19:00:00,Fortaleza,Bahia,2,1,H,...,3.0,3.5,0.761905,0.333333,0.366667,-0.5,0.428571,2.247616,3.287614,4.010213
757,Brazil,Serie A,2019,2019-12-08,19:00:00,Goias,Gremio,3,2,H,...,3.5,4.5,0.523810,0.571429,-0.433333,-1.0,-0.047619,2.810118,3.504733,2.791993
758,Brazil,Serie A,2019,2019-12-08,19:00:00,Internacional,Atletico-MG,2,1,H,...,4.0,4.5,0.333333,0.238095,-0.033333,-0.5,0.095238,2.234028,3.604519,3.656061


In [ ]:
comparacao

,Home,Away,Res,AvgH,AvgD,AvgA,Odds_Home,Odds_Draw,Odds_Away
0,Cruzeiro,Gremio,A,1.98,3.16,4.15,2.162352,3.580250,3.895542
1,Vitoria,Flamengo RJ,D,3.40,3.31,2.14,4.404367,2.924321,2.321494
2,Santos,Ceara,H,1.55,3.88,6.12,1.694127,4.006874,6.250424
3,America MG,Sport Recife,H,1.96,3.27,4.01,2.481997,3.535704,3.195530
4,Corinthians,Fluminense,H,1.62,3.55,6.03,1.909678,3.495467,5.270161
...,...,...,...,...,...,...,...,...,...
755,CSA,Sao Paulo,A,3.43,3.21,2.17,3.178737,3.727192,2.400569
756,Fortaleza,Bahia,H,2.02,3.51,3.54,2.247616,3.287614,4.010213
757,Goias,Gremio,H,2.01,3.52,3.53,2.810118,3.504733,2.791993
758,Internacional,Atletico-MG,H,1.57,4.09,5.42,2.234028,3.604519,3.656061


In [ ]:
comparacao2[(comparacao2['AvgH']> comparacao2['Odds_Home']+0.4) & (comparacao2['AvgH']>1.7) & (comparacao2['AvgH']<2)]

,Home,Away,Res,AvgH,AvgD,AvgA,Odds_Home,Odds_Draw,Odds_Away
33,Internacional,Atletico GO,H,1.75,3.36,5.06,1.294317,6.364461,14.231389
112,Fluminense,Coritiba,H,1.99,3.14,4.06,1.489750,4.724641,8.543279
246,Ceara,Atletico GO,A,1.95,3.21,4.16,1.511874,4.719485,7.896433
288,Internacional,Fortaleza,H,1.91,3.07,4.63,1.501623,4.752213,8.091484
354,Athletico-PR,Atletico GO,H,1.87,3.37,4.23,1.390750,5.409980,10.404724
498,Gremio,America MG,D,1.92,3.26,4.28,1.502123,4.786990,7.978557
584,Ceara,Chapecoense-SC,H,1.79,3.37,4.88,1.372222,5.389691,11.667632
627,Fortaleza,Gremio,H,1.84,3.34,4.44,1.382115,5.513630,10.516000
931,Athletico-PR,Atletico GO,H,1.97,3.26,4.08,1.490780,4.759157,8.399725
1039,Internacional,Bragantino,D,1.89,3.44,4.18,1.451223,4.886342,9.412287


In [ ]:
comparacao[(comparacao['AvgH']> comparacao['Odds_Home']+0.3) & (comparacao['AvgH']>1.7) & (comparacao['AvgH']<2)]

,Home,Away,Res,AvgH,AvgD,AvgA,Odds_Home,Odds_Draw,Odds_Away
191,Sport Recife,America MG,A,1.92,3.23,4.27,1.608940,4.165534,7.231933
304,Chapecoense-SC,America MG,H,1.98,3.10,4.25,1.615412,4.271971,6.814875
366,Atletico-PR,Ceara,D,1.86,3.23,4.57,1.538466,4.539765,7.714539
471,Fortaleza,Avai,H,1.96,3.09,4.34,1.657053,4.264488,6.178067
492,Internacional,Ceara,H,1.94,3.18,4.26,1.592992,4.391499,6.924752
515,Atletico-MG,Fluminense,H,1.80,3.60,4.29,1.471958,4.789379,8.944309
717,Santos,Cruzeiro,H,1.84,3.44,4.35,1.377731,5.390077,11.282368


In [ ]:
comparacao2[(comparacao2['AvgH']> comparacao2['Odds_Home']) & (comparacao2['AvgH']>1.7) & (comparacao2['AvgH']<2)].Res.value_counts()

H    67
D    26
A    10
Name: Res, dtype: int64

In [ ]:
df.Res.value_counts()

H    2082
D    1157
A    1040
        1
Name: Res, dtype: int64

In [ ]:
team_levels_2023 = {
    "Botafogo RJ": 3.5,
    "Palmeiras": 5,
    "Flamengo RJ": 4.5,
    "Atletico-MG": 4.5,
    "Fluminense": 4.5,
    "Gremio": 4,
    "Athletico-PR": 4,
    "Sao Paulo": 3.5,
    "Cruzeiro": 3,
    "Internacional": 3.5,
    "Santos": 3,
    "Fortaleza": 4,
    "Bragantino": 3.5,
    "Cuiaba": 2.5,
    "Bahia": 3,
    "Corinthians": 3.5,
    "America MG": 2.5,
    "Vasco": 2.5,
    "Coritiba": 2,
    "Goias": 2.5
}

team_levels_2023_rodada19 = {
    "Botafogo RJ": 3.5,
    "Palmeiras": 5,
    "Flamengo RJ": 4.5,
    "Atletico-MG": 4.5,
    "Fluminense": 4.5,
    "Gremio": 4,
    "Athletico-PR": 4,
    "Sao Paulo": 3.5,
    "Cruzeiro": 3,
    "Internacional": 3.5,
    "Santos": 3,
    "Fortaleza": 4,
    "Bragantino": 3.5,
    "Cuiaba": 2.5,
    "Bahia": 3,
    "Corinthians": 3.5,
    "America MG": 2.5,
    "Vasco": 3,
    "Coritiba": 2,
    "Goias": 2.5
}


In [ ]:
# Campeonato Brasileiro 2023
df2023 = df[df['Season'] == 2023]
df2023['Date'] = pd.to_datetime(df2023['Date'], format='%d/%m/%Y')
df2023 = df2023.sort_values('Date')

def last_10_points(team, date):
    last_10 = df2023[(df2023['Date'] < date) & ((df2023['Home'] == team) | (df2023['Away'] == team))][-10:]
    points = sum([3 if ((row['Home'] == team and row['Res'] == 'H') or (row['Away'] == team and row['Res'] == 'A')) else (1 if row['Res'] == 'D' else 0) for _, row in last_10.iterrows()])
    return points

df2023['points_home'] = df2023.apply(lambda row: last_10_points(row['Home'], row['Date']), axis=1)
df2023['points_away'] = df2023.apply(lambda row: last_10_points(row['Away'], row['Date']), axis=1)

df2023['perc_points_home_team'] = df2023['points_home'] / 30
df2023['perc_points_away_team'] = df2023['points_away'] / 30

df2023 = df2023.drop(['points_home', 'points_away'], axis=1)

team_list = list(set(df2023['Home'].unique()) | set(df2023['Away'].unique()))

df2023['level_team_home'] = 0
df2023['level_team_away'] = 0

for team in team_list:
    team_rows = df2023.loc[((df2023['Home'] == team) | (df2023['Away'] == team))].iloc[:19]
    df2023.loc[((df2023['Home'] == team) & (df2023.index.isin(team_rows.index))), 'level_team_home'] = team_levels_2023[team]
    df2023.loc[((df2023['Away'] == team) & (df2023.index.isin(team_rows.index))), 'level_team_away'] = team_levels_2023[team]
    team_rows = df2023.loc[((df2023['Home'] == team) | (df2023['Away'] == team))].iloc[19:]
    df2023.loc[((df2023['Home'] == team) & (df2023.index.isin(team_rows.index))), 'level_team_home'] = team_levels_2023_rodada19[team]
    df2023.loc[((df2023['Away'] == team) & (df2023.index.isin(team_rows.index))), 'level_team_away'] = team_levels_2023_rodada19[team]

def last_7_points(team, date, location):
    last_7 = df2023[(df2023['Date'] < date) & (((df2023['Home'] == team) & (location == 'home')) | ((df2023['Away'] == team) & (location == 'away')))][-7:]
    points = sum([3 if ((row['Home'] == team and row['Res'] == 'H') or (row['Away'] == team and row['Res'] == 'A')) else (1 if row['Res'] == 'D' else 0) for _, row in last_7.iterrows()])
    return points / 21

df2023['last_7_home'] = df2023.apply(lambda row: last_7_points(row['Home'], row['Date'], 'home'), axis=1)
df2023['last_7_away'] = df2023.apply(lambda row: last_7_points(row['Away'], row['Date'], 'away'), axis=1)


<ipython-input-90-6ea006d64fe0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2023['Date'] = pd.to_datetime(df2023['Date'], format='%d/%m/%Y')


In [ ]:
df2023['perc_points_diff'] =df2023['perc_points_home_team']-df2023['perc_points_away_team']
df2023['level_team_diff'] =df2023['level_team_home']-df2023['level_team_away']
df2023['last_7_diff'] =df2023['last_7_home']-df2023['last_7_away']

In [ ]:
X_2023 = df2023[['perc_points_diff','level_team_diff','last_7_diff']]

In [ ]:
y_pred_proba = model.predict_proba(X_2023)

# Extrair as probabilidades previstas para cada classe
proba_home = y_pred_proba[:, 0]
proba_draw = y_pred_proba[:, 1]
proba_away = y_pred_proba[:, 2]

# Calcular as odds
odds_home = 1 / proba_away
odds_draw = 1 / proba_draw
odds_away = 1 / proba_home

# Adicionar as odds ao dataframe original
df2023['Odds_Home'] = odds_home
df2023['Odds_Draw'] = odds_draw
df2023['Odds_Away'] = odds_away

In [ ]:
df2023[['Home',	'HG','AG','Away','Res','AvgH','AvgD','AvgA','Odds_Home', 'Odds_Draw','Odds_Away']].tail(50)

,Home,HG,AG,Away,Res,AvgH,AvgD,AvgA,Odds_Home,Odds_Draw,Odds_Away
4240,Bahia,1,1,Goias,D,1.70,3.66,5.09,1.858077,3.624752,5.392227
4241,America MG,2,1,Fortaleza,H,2.87,3.32,2.44,3.116676,3.103708,2.806244
4242,Botafogo RJ,1,0,Fluminense,H,2.70,3.25,2.62,2.501812,3.526391,3.171637
4243,Bragantino,2,0,Athletico-PR,H,1.83,3.57,4.26,2.294767,3.551940,3.552066
4244,Coritiba,1,2,Atletico-MG,A,4.60,3.36,1.83,4.474150,3.741784,1.964415
4245,Sao Paulo,4,2,Vasco,H,1.84,3.38,4.51,1.699303,4.057856,6.063741
4247,Flamengo RJ,1,0,Corinthians,H,1.41,4.40,8.09,1.644779,4.253737,6.378649
4248,Gremio,3,1,Internacional,H,2.02,3.26,3.85,1.849970,3.673661,5.354096
4246,Santos,0,0,Palmeiras,D,5.10,3.53,1.73,3.550846,3.165005,2.487099
4249,Cruzeiro,0,1,Cuiaba,A,1.58,3.75,6.27,1.937736,3.664426,4.751179


In [ ]:
df2023[['Home',	'HG','AG','Away','Res','AvgH','AvgD','AvgA','Odds_Home', 'Odds_Draw','Odds_Away']].head(50)

,Home,HG,AG,Away,Res,AvgH,AvgD,AvgA,Odds_Home,Odds_Draw,Odds_Away
4180,America MG,0,3,Fluminense,A,2.49,3.23,2.87,3.404066,3.038466,2.654742
4181,Palmeiras,2,1,Cuiaba,H,1.35,4.92,8.54,1.299576,6.409700,13.422331
4182,Athletico-PR,2,0,Goias,H,1.62,3.57,6.17,1.517588,4.791908,7.556771
4183,Botafogo RJ,2,1,Sao Paulo,H,2.40,3.05,3.15,2.021965,3.515032,4.548107
4184,Bragantino,2,1,Bahia,H,1.84,3.56,4.21,1.909678,3.495467,5.270161
4185,Fortaleza,1,1,Internacional,D,1.91,3.44,4.07,1.909678,3.495467,5.270161
4186,Atletico-MG,1,2,Vasco,A,1.80,3.50,4.54,1.395523,5.488868,9.878914
4187,Corinthians,2,1,Cruzeiro,H,1.77,3.41,4.96,1.909678,3.495467,5.270161
4188,Flamengo RJ,3,0,Coritiba,H,1.38,4.76,8.13,1.299576,6.409700,13.422331
4189,Gremio,1,0,Santos,H,1.77,3.38,4.99,1.694127,4.006874,6.250424


In [ ]:
df2023[['Season','Date','Home',	'HG','AG','Away','Res','AvgH','AvgD','AvgA','Odds_Home', 'Odds_Draw','Odds_Away']].to_excel('brasileiro2023.xlsx', index=False)

In [ ]:
validacao_atual[['Season','Date','Home',	'Away','Res','AvgH','AvgD','AvgA','Odds_Home', 'Odds_Draw','Odds_Away']].to_excel('brasileiro-20-21-22.xlsx', index=False)

In [ ]:
df2023[['Season','Date','Home',	'HG','AG','Away','Res','AvgH','AvgD','AvgA','Odds_Home', 'Odds_Draw','Odds_Away']].tail(10)

,Season,Date,Home,HG,AG,Away,Res,AvgH,AvgD,AvgA,Odds_Home,Odds_Draw,Odds_Away
4280,2023,2023-06-21,Cruzeiro,,,Fortaleza,,2.05,3.25,3.60,2.516360,3.567372,3.115744
4281,2023,2023-06-21,Sao Paulo,NaN,NaN,Athletico-PR,NaN,1.80,3.40,4.33,2.353465,3.686647,3.303625
4282,2023,2023-06-21,Santos,NaN,NaN,Corinthians,NaN,2.37,3.10,3.10,2.330923,3.660624,3.373302
4283,2023,2023-06-21,Bahia,NaN,NaN,Palmeiras,NaN,4.50,3.60,1.75,3.846484,3.349156,2.266830
4284,2023,2023-06-21,Fluminense,NaN,NaN,Atletico-MG,NaN,2.20,3.20,3.25,2.009543,3.512777,4.615350
4288,2023,2023-06-22,Vasco,NaN,NaN,Goias,NaN,1.80,3.50,4.50,1.997331,3.510758,4.683900
4285,2023,2023-06-22,Gremio,NaN,NaN,America MG,NaN,1.65,3.80,5.00,1.487397,4.776815,8.452827
4286,2023,2023-06-22,Coritiba,NaN,NaN,Internacional,NaN,3.00,3.20,2.30,3.230080,3.600954,2.426171
4287,2023,2023-06-22,Cuiaba,NaN,NaN,Botafogo RJ,NaN,2.87,3.20,2.45,2.708333,3.240365,3.110815
4289,2023,2023-06-22,Bragantino,NaN,NaN,Flamengo RJ,NaN,3.10,3.40,2.20,2.812373,3.810599,2.624780


In [ ]:
df2023[['Season','Date','Home',	'HG','AG','Away','Res','AvgH','AvgD','AvgA','Odds_Home', 'Odds_Draw','Odds_Away']].tail(10)

,Season,Date,Home,HG,AG,Away,Res,AvgH,AvgD,AvgA,Odds_Home,Odds_Draw,Odds_Away
4280,2023,2023-06-21,Cruzeiro,,,Fortaleza,,2.05,3.25,3.60,2.516360,3.567372,3.115744
4281,2023,2023-06-21,Sao Paulo,NaN,NaN,Athletico-PR,NaN,1.80,3.40,4.33,2.353465,3.686647,3.303625
4282,2023,2023-06-21,Santos,NaN,NaN,Corinthians,NaN,2.37,3.10,3.10,2.330923,3.660624,3.373302
4283,2023,2023-06-21,Bahia,NaN,NaN,Palmeiras,NaN,4.50,3.60,1.75,3.846484,3.349156,2.266830
4284,2023,2023-06-21,Fluminense,NaN,NaN,Atletico-MG,NaN,2.20,3.20,3.25,2.009543,3.512777,4.615350
4288,2023,2023-06-22,Vasco,NaN,NaN,Goias,NaN,1.80,3.50,4.50,1.997331,3.510758,4.683900
4285,2023,2023-06-22,Gremio,NaN,NaN,America MG,NaN,1.65,3.80,5.00,1.487397,4.776815,8.452827
4286,2023,2023-06-22,Coritiba,NaN,NaN,Internacional,NaN,3.00,3.20,2.30,3.230080,3.600954,2.426171
4287,2023,2023-06-22,Cuiaba,NaN,NaN,Botafogo RJ,NaN,2.87,3.20,2.45,2.708333,3.240365,3.110815
4289,2023,2023-06-22,Bragantino,NaN,NaN,Flamengo RJ,NaN,3.10,3.40,2.20,2.812373,3.810599,2.624780


In [ ]:
df2023[['Season','Date','Home','Away','AvgH','AvgD','AvgA','Odds_Home', 'Odds_Draw','Odds_Away']].tail(10)

,Season,Date,Home,Away,AvgH,AvgD,AvgA,Odds_Home,Odds_Draw,Odds_Away
4280,2023,2023-06-21,Cruzeiro,Fortaleza,2.05,3.25,3.60,2.516360,3.567372,3.115744
4281,2023,2023-06-21,Sao Paulo,Athletico-PR,1.80,3.40,4.33,2.353465,3.686647,3.303625
4282,2023,2023-06-21,Santos,Corinthians,2.37,3.10,3.10,2.330923,3.660624,3.373302
4283,2023,2023-06-21,Bahia,Palmeiras,4.50,3.60,1.75,3.846484,3.349156,2.266830
4284,2023,2023-06-21,Fluminense,Atletico-MG,2.20,3.20,3.25,2.009543,3.512777,4.615350
4288,2023,2023-06-22,Vasco,Goias,1.80,3.50,4.50,1.997331,3.510758,4.683900
4285,2023,2023-06-22,Gremio,America MG,1.65,3.80,5.00,1.487397,4.776815,8.452827
4286,2023,2023-06-22,Coritiba,Internacional,3.00,3.20,2.30,3.230080,3.600954,2.426171
4287,2023,2023-06-22,Cuiaba,Botafogo RJ,2.87,3.20,2.45,2.708333,3.240365,3.110815
4289,2023,2023-06-22,Bragantino,Flamengo RJ,3.10,3.40,2.20,2.812373,3.810599,2.624780


In [ ]:
df2023[['Season','Date','Home',	'HG','AG','Away','Res','AvgH','AvgD','AvgA','Odds_Home', 'Odds_Draw','Odds_Away']].head(100).to_excel('teste2.xlsx',index=False)

Equador

In [ ]:
equador_levels_2019 = {
    "Macara": 5,
    "Barcelona SC": 5,
    "U. Catolica": 4.5,
    "Ind. del Valle":5,
    "Delfin": 4.5,
    "LDU Quito": 4.5,
    "Aucas": 3.5,
    "Emelec":4,
    "EL Nacional":2.5,
    "Dep. Cuenca":3.5,
    "Olmedo":3.5,
    "Guayaquil City":3,
    "Mushuc Runa":2.5,
    "Tecnico U.":2.5,
    "America de Quito":2.5,
    "Fuerza A.":2
}

equador_levels_2019_rodada15 = {
    "Macara": 5,
    "Barcelona SC": 5,
    "U. Catolica": 4.5,
    "Ind. del Valle":4.5,
    "Delfin": 4.5,
    "LDU Quito": 4,
    "Aucas": 4,
    "Emelec":4,
    "EL Nacional":3.5,
    "Dep. Cuenca":3,
    "Olmedo":3,
    "Guayaquil City":3,
    "Mushuc Runa":2.5,
    "Tecnico U.":2,
    "America de Quito":2,
    "Fuerza A.":1.5
}




In [ ]:
df

,Country,League,Season,Date,Time,Home,Away,HG,AG,Res,PH,PD,PA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA
0,Brazil,Serie A,2012,2012-05-19,22:30:00,Palmeiras,Portuguesa,1,1,D,1.75,3.86,5.25,1.76,3.87,5.31,1.69,3.50,4.90
1,Brazil,Serie A,2012,2012-05-19,22:30:00,Sport Recife,Flamengo RJ,1,1,D,2.83,3.39,2.68,2.83,3.42,2.7,2.59,3.23,2.58
2,Brazil,Serie A,2012,2012-05-20,01:00:00,Figueirense,Nautico,2,1,H,1.6,4.04,6.72,1.67,4.05,7.22,1.59,3.67,5.64
3,Brazil,Serie A,2012,2012-05-20,20:00:00,Botafogo RJ,Sao Paulo,4,2,H,2.49,3.35,3.15,2.49,3.39,3.15,2.35,3.26,2.84
4,Brazil,Serie A,2012,2012-05-20,20:00:00,Corinthians,Fluminense,0,1,A,1.96,3.53,4.41,1.96,3.53,4.41,1.89,3.33,3.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4285,Brazil,Serie A,2023,2023-06-22,19:00:00,Gremio,America MG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.65,3.80,5.00
4286,Brazil,Serie A,2023,2023-06-22,20:00:00,Coritiba,Internacional,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.00,3.20,2.30
4287,Brazil,Serie A,2023,2023-06-22,20:00:00,Cuiaba,Botafogo RJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.87,3.20,2.45
4288,Brazil,Serie A,2023,2023-06-22,20:00:00,Vasco,Goias,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.80,3.50,4.50


In [ ]:
equador = pd.read_excel('/content/equador2019.xlsx')

In [ ]:
equador

,Date,TeamHome,TeamAway,GolHome,GolAway,OddHome,OddDraw,OddAway
0,2019-02-09,Fuerza A.,Delfin,0,2,2.82,3.29,2.31
1,2019-02-09,Guayaquil City,Macara,1,1,3.03,3.20,2.28
2,2019-02-09,LDU Quito,Olmedo,3,2,1.39,4.25,7.35
3,2019-02-10,Aucas,Emelec,0,2,2.35,3.02,2.96
4,2019-02-10,Barcelona SC,EL Nacional,5,2,1.35,4.56,7.65
...,...,...,...,...,...,...,...,...
234,2019-11-03,Guayaquil City,Dep. Cuenca,1,2,2.14,3.49,2.98
235,2019-11-03,Emelec,Mushuc Runa,6,1,1.12,7.78,16.52
236,2019-11-03,Ind. del Valle,Delfin,0,0,2.26,3.35,2.91
237,2019-11-03,Olmedo,Barcelona SC,1,2,2.68,3.34,2.39


In [ ]:
novo_nome_colunas = {'Date': 'Date',
                     'TeamHome': 'Home',
                     'TeamAway': 'Away',
                     'GolHome': "HG",
                     "GolAway": "AG",
                     "OddHome": "AvgH",
                     "OddDraw": "AvgD",
                     "OddAway" : "AvgA"
                     }

In [ ]:
equador = equador.rename(columns=novo_nome_colunas)

In [ ]:
# Função para atribuir o valor correto a cada linha
def atribuir_res(row):
    if row['HG'] > row['AG']:
        return 'H'
    elif row['HG'] == row['AG']:
        return 'D'
    else:
        return 'A'

# Aplicando a função em cada linha para criar a nova coluna 'Res'
equador['Res'] = df.apply(atribuir_res, axis=1)

In [ ]:
# Campeonato Brasileiro 2023
equador['Date'] = pd.to_datetime(equador['Date'], format='%d/%m/%Y')
equador = equador.sort_values('Date')

def last_10_points(team, date):
    last_10 = equador[(equador['Date'] < date) & ((equador['Home'] == team) | (equador['Away'] == team))][-10:]
    points = sum([3 if ((row['Home'] == team and row['Res'] == 'H') or (row['Away'] == team and row['Res'] == 'A')) else (1 if row['Res'] == 'D' else 0) for _, row in last_10.iterrows()])
    return points

equador['points_home'] = equador.apply(lambda row: last_10_points(row['Home'], row['Date']), axis=1)
equador['points_away'] = equador.apply(lambda row: last_10_points(row['Away'], row['Date']), axis=1)

equador['perc_points_home_team'] = equador['points_home'] / 30
equador['perc_points_away_team'] = equador['points_away'] / 30

equador = equador.drop(['points_home', 'points_away'], axis=1)

team_list = list(set(equador['Home'].unique()) | set(equador['Away'].unique()))

df2023['level_team_home'] = 0
df2023['level_team_away'] = 0

for team in team_list:
    team_rows = equador.loc[((equador['Home'] == team) | (equador['Away'] == team))].iloc[:19]
    equador.loc[((equador['Home'] == team) & (equador.index.isin(team_rows.index))), 'level_team_home'] = equador_levels_2019[team]
    equador.loc[((equador['Away'] == team) & (equador.index.isin(team_rows.index))), 'level_team_away'] = equador_levels_2019[team]
    team_rows = equador.loc[((equador['Home'] == team) | (equador['Away'] == team))].iloc[19:]
    equador.loc[((equador['Home'] == team) & (equador.index.isin(team_rows.index))), 'level_team_home'] = equador_levels_2019_rodada15[team]
    equador.loc[((equador['Away'] == team) & (equador.index.isin(team_rows.index))), 'level_team_away'] = equador_levels_2019_rodada15[team]

def last_7_points(team, date, location):
    last_7 = equador[(equador['Date'] < date) & (((equador['Home'] == team) & (location == 'home')) | ((equador['Away'] == team) & (location == 'away')))][-7:]
    points = sum([3 if ((row['Home'] == team and row['Res'] == 'H') or (row['Away'] == team and row['Res'] == 'A')) else (1 if row['Res'] == 'D' else 0) for _, row in last_7.iterrows()])
    return points / 21

equador['last_7_home'] = equador.apply(lambda row: last_7_points(row['Home'], row['Date'], 'home'), axis=1)
equador['last_7_away'] = equador.apply(lambda row: last_7_points(row['Away'], row['Date'], 'away'), axis=1)

In [ ]:
equador.Res.value_counts()

H    117
D     64
A     58
Name: Res, dtype: int64

In [ ]:
df2022.Res.value_counts()

H    168
D    108
A    104
Name: Res, dtype: int64

In [ ]:
equador

,Date,Home,Away,HG,AG,AvgH,AvgD,AvgA,Res,perc_points_home_team,perc_points_away_team,level_team_home,level_team_away,last_7_home,last_7_away
0,2019-02-09,Fuerza A.,Delfin,0,2,2.82,3.29,2.31,D,0.000000,0.000000,2.0,4.5,0.000000,0.000000
1,2019-02-09,Guayaquil City,Macara,1,1,3.03,3.20,2.28,D,0.000000,0.000000,3.0,5.0,0.000000,0.000000
2,2019-02-09,LDU Quito,Olmedo,3,2,1.39,4.25,7.35,H,0.000000,0.000000,4.5,3.5,0.000000,0.000000
3,2019-02-10,Aucas,Emelec,0,2,2.35,3.02,2.96,H,0.000000,0.000000,3.5,4.0,0.000000,0.000000
4,2019-02-10,Barcelona SC,EL Nacional,5,2,1.35,4.56,7.65,A,0.000000,0.000000,5.0,2.5,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,2019-11-03,Ind. del Valle,Delfin,0,0,2.26,3.35,2.91,H,0.400000,0.400000,4.5,4.5,0.523810,0.238095
237,2019-11-03,Olmedo,Barcelona SC,1,2,2.68,3.34,2.39,D,0.333333,0.500000,3.0,5.0,0.619048,0.428571
234,2019-11-03,Guayaquil City,Dep. Cuenca,1,2,2.14,3.49,2.98,H,0.366667,0.500000,3.0,3.0,0.523810,0.142857
235,2019-11-03,Emelec,Mushuc Runa,6,1,1.12,7.78,16.52,H,0.533333,0.466667,4.0,2.5,0.809524,0.428571


In [ ]:
equador['perc_points_diff'] =equador['perc_points_home_team']-equador['perc_points_away_team']
equador['level_team_diff'] =equador['level_team_home']-equador['level_team_away']
equador['last_7_diff'] =equador['last_7_home']-equador['last_7_away']

In [ ]:
Xequador = equador[['perc_points_diff','level_team_diff','last_7_diff']]

In [ ]:
y_pred_proba = model.predict_proba(Xequador)

# Extrair as probabilidades previstas para cada classe
proba_home = y_pred_proba[:, 0]
proba_draw = y_pred_proba[:, 1]
proba_away = y_pred_proba[:, 2]

# Calcular as odds
odds_home = 1 / proba_away
odds_draw = 1 / proba_draw
odds_away = 1 / proba_home

# Adicionar as odds ao dataframe original
equador['Odds_Home'] = odds_home
equador['Odds_Draw'] = odds_draw
equador['Odds_Away'] = odds_away

In [ ]:
equador

,Date,Home,Away,HG,AG,AvgH,AvgD,AvgA,Res,perc_points_home_team,...,level_team_home,level_team_away,last_7_home,last_7_away,perc_points_diff,level_team_diff,last_7_diff,Odds_Home,Odds_Draw,Odds_Away
0,2019-02-09,Fuerza A.,Delfin,0,2,2.82,3.29,2.31,D,0.000000,...,2.0,4.5,0.000000,0.000000,0.000000,-2.5,0.000000,4.404367,2.924321,2.321494
1,2019-02-09,Guayaquil City,Macara,1,1,3.03,3.20,2.28,D,0.000000,...,3.0,5.0,0.000000,0.000000,0.000000,-2.0,0.000000,3.404066,3.038466,2.654742
2,2019-02-09,LDU Quito,Olmedo,3,2,1.39,4.25,7.35,H,0.000000,...,4.5,3.5,0.000000,0.000000,0.000000,1.0,0.000000,1.694127,4.006874,6.250424
3,2019-02-10,Aucas,Emelec,0,2,2.35,3.02,2.96,H,0.000000,...,3.5,4.0,0.000000,0.000000,0.000000,-0.5,0.000000,2.162352,3.580250,3.895542
4,2019-02-10,Barcelona SC,EL Nacional,5,2,1.35,4.56,7.65,A,0.000000,...,5.0,2.5,0.000000,0.000000,0.000000,2.5,0.000000,1.299576,6.409700,13.422331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,2019-11-03,Ind. del Valle,Delfin,0,0,2.26,3.35,2.91,H,0.400000,...,4.5,4.5,0.523810,0.238095,0.000000,0.0,0.285714,1.943537,3.727444,4.623262
237,2019-11-03,Olmedo,Barcelona SC,1,2,2.68,3.34,2.39,D,0.333333,...,3.0,5.0,0.619048,0.428571,-0.166667,-2.0,0.190476,3.597699,3.192817,2.447883
234,2019-11-03,Guayaquil City,Dep. Cuenca,1,2,2.14,3.49,2.98,H,0.366667,...,3.0,3.0,0.523810,0.142857,-0.133333,0.0,0.380952,1.947181,3.717952,4.612765
235,2019-11-03,Emelec,Mushuc Runa,6,1,1.12,7.78,16.52,H,0.533333,...,4.0,2.5,0.809524,0.428571,0.066667,1.5,0.380952,1.468132,4.765927,9.173954


In [ ]:
equador[['Date','Home','Away','Res','AvgH','AvgD','AvgA','Odds_Home', 'Odds_Draw','Odds_Away']].head(60)

,Date,Home,Away,Res,AvgH,AvgD,AvgA,Odds_Home,Odds_Draw,Odds_Away
0,2019-02-09,Fuerza A.,Delfin,D,2.82,3.29,2.31,4.404367,2.924321,2.321494
1,2019-02-09,Guayaquil City,Macara,D,3.03,3.20,2.28,3.404066,3.038466,2.654742
2,2019-02-09,LDU Quito,Olmedo,H,1.39,4.25,7.35,1.694127,4.006874,6.250424
3,2019-02-10,Aucas,Emelec,H,2.35,3.02,2.96,2.162352,3.580250,3.895542
4,2019-02-10,Barcelona SC,EL Nacional,A,1.35,4.56,7.65,1.299576,6.409700,13.422331
5,2019-02-10,Dep. Cuenca,Ind. del Valle,H,2.38,3.16,2.80,2.831758,3.374255,2.859597
6,2019-02-11,Tecnico U.,U. Catolica,A,3.50,3.48,1.93,3.404066,3.038466,2.654742
7,2019-02-12,America de Quito,Mushuc Runa,D,2.26,3.19,3.02,2.021965,3.515032,4.548107
8,2019-02-16,EL Nacional,Aucas,D,3.65,3.57,1.87,2.481997,3.535704,3.195530
9,2019-02-16,Olmedo,America de Quito,H,2.26,3.16,3.01,1.687714,4.003552,6.347470


In [ ]:
equador[['Date','Home','Away','Res','AvgH','AvgD','AvgA','Odds_Home', 'Odds_Draw','Odds_Away']].to_excel('equador2019validacao.xlsx', index= False)